In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm  # For progress bars
import numpy as np


model_name = "distilgpt2"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model = model.to("cuda")  # Move to GPU

/home/thomasdev/anaconda3/envs/ultralytics-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/thomasdev/anaconda3/envs/ultralytics-env/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/thomasdev/anaconda3/envs/ultralytics-env/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:


# Input text
text = "Shut the fuck"

# Tokenize and move to the same device as the model
encoded_input = tokenizer(text, return_tensors='pt')
# Move each tensor to the GPU
encoded_input = {k: v.to("cuda") for k, v in encoded_input.items()}

# Run the model
with torch.no_grad():  # Add this for inference to save memory
    output = model(**encoded_input)

# Print some output info
print(f"Output shape: {output.logits.shape}")
print(f"Device of output: {output.logits.device}")



Output shape: torch.Size([1, 3, 50257])
Device of output: cuda:0


In [ ]:
genereted_token=0
text = "Shut the fuck"

while genereted_token != tokenizer.encode(tokenizer.special_tokens_map['eos_token'])[0]:
    encoded_input = tokenizer(text, return_tensors='pt')
    # Move each tensor to the GPU
    encoded_input_gpu = {k: v.to("cuda") for k, v in encoded_input.items()}
    with torch.no_grad():  # Add this for inference to save memory
        output = model(**encoded_input)
    torch.argmax(output.logits[:,-1,:], dim=-1).item()
    

In [17]:
tokenizer.encode(tokenizer.special_tokens_map['eos_token'])[0]

50256

In [26]:
tokenizer(text, return_tensors='pt')


{'input_ids': tensor([[39079,   262,  5089]]), 'attention_mask': tensor([[1, 1, 1]])}

In [37]:
def generate_until_end_token(model, tokenizer, start_text="", max_length=100, 
                           temperature=0.7, device="cuda"):
    """
    Generate text using a while loop until the end token is produced or max_length is reached.
    
    Args:
        model: The GPT model to use for generation
        tokenizer: The tokenizer to use
        start_text: Optional starting text prompt
        max_length: Maximum number of tokens to generate
        temperature: Controls randomness in generation (higher = more random, lower = more deterministic)
        device: Device to run generation on ('cuda' or 'cpu')
        
    Returns:
        The generated text
    """
    # Ensure model is on the right device
    model = model.to(device)
    
    # Tokenize the start text
    if start_text:
        input_ids = tokenizer.encode(start_text, return_tensors='pt').to(device)
    else:
        # If no start text, begin with BOS token
        input_ids = torch.tensor([[tokenizer.bos_token_id]], device=device)
    
    # Get the end token ID
    end_token_id = tokenizer.eos_token_id
    
    # Track tokens generated
    tokens_generated = 0
    found_end_token = False
    
    print(f"Generating with temperature={temperature}")
    
    # Generate until we find the end token or reach max_length
    while tokens_generated < max_length:
        # Forward pass through the model
        with torch.no_grad():
            outputs = model(input_ids=input_ids)
        
        # Get the next token prediction (last position)
        next_token_logits = outputs.logits[:, -1, :]
        
        # Apply temperature scaling
        if temperature == 0:
            # Use greedy sampling (argmax) for temperature 0
            next_token_id = torch.argmax(next_token_logits, dim=-1).item()
        else:
            # Apply temperature scaling
            scaled_logits = next_token_logits / temperature
            
            # Convert to probabilities
            probs = torch.softmax(scaled_logits, dim=-1)
            
            # Sample from the distribution
            next_token_id = torch.multinomial(probs, num_samples=1).item()
        
        # Decode and print the token
        token_text = tokenizer.decode([next_token_id])
        print(f"  Token {tokens_generated+1}: '{token_text}' (ID: {next_token_id})")
        
        # Check if we've generated the end token
        if next_token_id == end_token_id:
            print(f"Found end token! Generation complete after {tokens_generated+1} tokens.")
            found_end_token = True
            break
        
        # Update input_ids to include the new token (preserve full context)
        input_ids = torch.cat([input_ids, torch.tensor([[next_token_id]], device=device)], dim=1)
        tokens_generated += 1
    
    if not found_end_token:
        print(f"Reached maximum length ({max_length}) without finding end token.")
    
    # Get all generated tokens (including the starting context)
    all_tokens = input_ids[0].tolist()
    
    # Decode the full generated text
    full_text = tokenizer.decode(all_tokens)
    return full_text


In [40]:
# Example usage
if __name__ == "__main__":
    # Load model and tokenizer

    
    # Set device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    # Generate text
    start_prompt = "The AI will conquer us"
    generated_text = generate_until_end_token(
        model=model,
        tokenizer=tokenizer,
        start_text=start_prompt,
        max_length=100,
        temperature=0.8,
        device=device
    )
    
    print("\nFull generated text:")
    print(generated_text)

Using device: cuda
Generating with temperature=0.8
  Token 1: ' all' (ID: 477)
  Token 2: '.' (ID: 13)
  Token 3: '
' (ID: 198)
  Token 4: '
' (ID: 198)
  Token 5: 'Now' (ID: 3844)
  Token 6: ',' (ID: 11)
  Token 7: ' since' (ID: 1201)
  Token 8: ' the' (ID: 262)
  Token 9: ' AI' (ID: 9552)
  Token 10: ' has' (ID: 468)
  Token 11: ' an' (ID: 281)
  Token 12: ' amazing' (ID: 4998)
  Token 13: ' ability' (ID: 2694)
  Token 14: ' to' (ID: 284)
  Token 15: ' identify' (ID: 5911)
  Token 16: ' interesting' (ID: 3499)
  Token 17: ' areas' (ID: 3006)
  Token 18: ' or' (ID: 393)
  Token 19: ' places' (ID: 4113)
  Token 20: ',' (ID: 11)
  Token 21: ' it' (ID: 340)
  Token 22: ' all' (ID: 477)
  Token 23: ' seems' (ID: 2331)
  Token 24: ' to' (ID: 284)
  Token 25: ' be' (ID: 307)
  Token 26: ' going' (ID: 1016)
  Token 27: ' to' (ID: 284)
  Token 28: ' be' (ID: 307)
  Token 29: ' on' (ID: 319)
  Token 30: ' its' (ID: 663)
  Token 31: ' way' (ID: 835)
  Token 32: '.' (ID: 13)
  Token 33: ' We' (I

In [10]:
output.logits.shape

torch.Size([1, 3, 50257])

In [13]:
torch.argmax(output.logits[:,-1,:], dim=-1).item()

572

In [34]:
tokenizer.decode(torch.argmax(output.logits[:,-1,:], dim=-1).item())

' off'

In [30]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm  # For progress bars
import numpy as np


In [ ]:



def find_similar_tokens(model_name="distilgpt2", k=5, batch_size=1000, sample_size=20, 
                       context_aware=True, smooth_alpha=0.3):
    """
    Find the k most similar tokens for each token in the embedding table using cosine similarity.
    
    Args:
        model_name: Name of the pretrained model
        k: Number of similar tokens to find for each token
        batch_size: Process this many tokens at once (for memory efficiency)
        sample_size: Number of example tokens to display
        context_aware: If True, only return tokens with the same context
                       (e.g., both have or both lack the 'Ġ' prefix)
        smooth_alpha: Label smoothing hyperparameter for scaling similarity values
    
    Returns:
        - Dictionary mapping each token to its k most similar tokens
        - T matrix: shape [vocab_size, k] containing indices of similar tokens
        - V matrix: shape [vocab_size, k] containing normalized similarity values scaled by alpha
    """
    print(f"Loading model {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # Get the embedding table
    embedding_table = model.transformer.wte.weight.data  # Shape: [vocab_size, embed_dim]
    vocab_size, embed_dim = embedding_table.shape
    print(f"Vocabulary size: {vocab_size}, Embedding dimension: {embed_dim}")
    
    # 1. Normalize all embedding vectors to unit length
    print("Normalizing embedding vectors...")
    norms = torch.norm(embedding_table, p=2, dim=1, keepdim=True)
    normalized_embeddings = embedding_table / norms
    
    # Get vocabulary mapping for easier reference
    vocab = tokenizer.get_vocab()
    id_to_token = {id: token for token, id in vocab.items()}
    
    # Create lookup for tokens with/without Ġ prefix if using context_aware mode
    if context_aware:
        has_g_prefix = {}
        for token_id in range(vocab_size):
            token = id_to_token[token_id]
            has_g_prefix[token_id] = token.startswith('Ġ')
    
    # Initialize matrices T and V to store indices and values
    T = np.zeros((vocab_size, k), dtype=np.int32)  # Token indices
    V = np.zeros((vocab_size, k), dtype=np.float32)  # Similarity values
    
    # 2. Compute cosine similarity matrix and find top-k in batches
    print(f"Computing similarities and finding top {k} similar tokens...")
    most_similar = {}
    
    # Process in batches to avoid memory issues with large vocabularies
    for i in tqdm(range(0, vocab_size, batch_size)):
        end_idx = min(i + batch_size, vocab_size)
        batch_embeddings = normalized_embeddings[i:end_idx]
        
        # Compute similarity matrix for this batch
        batch_similarity = torch.mm(batch_embeddings, normalized_embeddings.T)  # [batch_size, vocab_size]
        
        # For each token in the batch, find the k most similar tokens
        for batch_idx, global_idx in enumerate(range(i, end_idx)):
            token = id_to_token[global_idx]
            similarities = batch_similarity[batch_idx].clone()
            
            # Set similarity with itself to -1 to exclude it
            similarities[global_idx] = -1
            
            if context_aware:
                # Get the prefix status of the current token
                has_prefix = has_g_prefix[global_idx]
                
                # Filter for tokens with matching prefix status
                filtered_indices = []
                filtered_values = []
                
                # Get more tokens than k initially, we'll filter them
                num_candidates = min(k * 10, vocab_size - 1)  # 10x more candidates to filter from
                candidate_indices = torch.topk(similarities, num_candidates).indices.tolist()
                candidate_values = torch.topk(similarities, num_candidates).values.tolist()
                
                # Filter candidates by matching prefix
                for idx, val in zip(candidate_indices, candidate_values):
                    if has_g_prefix[idx] == has_prefix:
                        filtered_indices.append(idx)
                        filtered_values.append(val)
                        if len(filtered_indices) >= k:
                            break
                
                # If we couldn't find enough matches, pad with the best remaining tokens
                if len(filtered_indices) < k:
                    # Add remaining highest similarity tokens regardless of prefix
                    for idx, val in zip(candidate_indices, candidate_values):
                        if idx not in filtered_indices:
                            filtered_indices.append(idx)
                            filtered_values.append(val)
                            if len(filtered_indices) >= k:
                                break
                
                top_k_indices = filtered_indices[:k]
                top_k_values = filtered_values[:k]
            else:
                # Original approach - get top k without filtering
                top_k_indices = torch.topk(similarities, k).indices.tolist()
                top_k_values = torch.topk(similarities, k).values.tolist()
            
            # Store in matrices T and V
            T[global_idx, :] = top_k_indices
            V[global_idx, :] = top_k_values
            
            # Store in dictionary (for display purposes)
            most_similar[token] = [(id_to_token[idx], float(val)) for idx, val in zip(top_k_indices, top_k_values)]
    
    # 3. Normalize and apply smoothing to V matrix
    print(f"Normalizing similarity values and applying smoothing with alpha={smooth_alpha}...")
    
    # Calculate row sums for normalization
    row_sums = np.sum(V, axis=1, keepdims=True)
    
    # Avoid division by zero (if all similarities are near zero)
    row_sums = np.maximum(row_sums, 1e-10)
    
    # Normalize each row to sum to 1
    V_normalized = V / row_sums
    
    # Apply smoothing factor
    V = V_normalized * smooth_alpha
    
    # Verify normalization
    row_sums_after = np.sum(V, axis=1) / smooth_alpha  # Divide by alpha to check if they sum to 1
    print(f"After normalization, average row sum (before alpha): {np.mean(row_sums_after):.6f}")
    
    # Print sample results
    print(f"\nShowing {sample_size} examples of similar tokens:")
    sample_tokens = list(most_similar.keys())[:sample_size]
    
    for token in sample_tokens:
        similar_tokens = most_similar[token]
        # Handle display of special characters
        printable_token = repr(token)[1:-1] if not token.isprintable() else token
        print(f"Token: '{printable_token}'")
        for sim_token, sim_score in similar_tokens:
            printable_sim = repr(sim_token)[1:-1] if not sim_token.isprintable() else sim_token
            print(f"  - '{printable_sim}': {sim_score:.4f}")
        print()
    
    # Print shape of matrices
    print(f"Shape of matrix T: {T.shape}")
    print(f"Shape of matrix V: {V.shape} (normalized and scaled by alpha={smooth_alpha})")
    
    return most_similar, T, V

# Example usage
if __name__ == "__main__":
    # Set device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    # Find similar tokens with smoothing
    similar_tokens, T, V = find_similar_tokens(
        model_name="distilgpt2",     # Change to your model
        k=5,                         # Number of similar tokens to find
        sample_size=10,              # Number of examples to display
        context_aware=True,          # Only match tokens with same prefix status
        smooth_alpha=0.3           # Label smoothing hyperparameter
    )
    
 
    


Using device: cuda
Loading model distilgpt2...
Vocabulary size: 50257, Embedding dimension: 768
Normalizing embedding vectors...
Computing similarities and finding top 5 similar tokens...


100%|██████████| 51/51 [00:24<00:00,  2.05it/s]


Normalizing similarity values and applying smoothing with alpha=0.3...
After normalization, average row sum (before alpha): 1.000000

Showing 10 examples of similar tokens:
Token: '!'
  - '!!': 0.7689
  - '!!!': 0.7591
  - '!"': 0.7241
  - '!.': 0.6954
  - '!'': 0.6777

Token: '"'
  - '",': 0.6725
  - ',"': 0.6493
  - '."': 0.6381
  - '"-': 0.6252
  - '".': 0.6019

Token: '#'
  - '##': 0.5825
  - '.#': 0.5645
  - '/#': 0.5181
  - '####': 0.4722
  - ':#': 0.4718

Token: '$'
  - '$,': 0.6443
  - '$.': 0.6241
  - 'Â£': 0.5739
  - '.$': 0.5255
  - '/$': 0.4997

Token: '%'
  - '%,': 0.7222
  - '%.': 0.6863
  - '%-': 0.6612
  - '%;': 0.6215
  - '%)': 0.6076

Token: '&'
  - '&&': 0.4763
  - '#': 0.4380
  - '/?': 0.4237
  - '(&': 0.4052
  - '%': 0.3807

Token: '''
  - '',': 0.6665
  - ''.': 0.6392
  - ''-': 0.5912
  - ''"': 0.5707
  - ''?': 0.5575

Token: '('
  - '("': 0.6288
  - '((': 0.6153
  - '(-': 0.5900
  - '('': 0.5718
  - '[': 0.5596

Token: ')'
  - '),': 0.7698
  - ').': 0.7322
  - '.

In [16]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [18]:
similar_tokens['<|endoftext|>']

[('Ċ', 0.5828453302383423),
 ('ĊĊ', 0.5247935652732849),
 ('In', 0.380106121301651),
 ('This', 0.3797813057899475),
 ('It', 0.37661904096603394)]

In [26]:
V

array([[0.06363054, 0.06281649, 0.05991939, 0.05755031, 0.05608328],
       [0.06330582, 0.06111777, 0.06006429, 0.05885125, 0.05666085],
       [0.06697471, 0.0649103 , 0.05957501, 0.05429798, 0.054242  ],
       ...,
       [0.07841571, 0.05786163, 0.05558842, 0.05424513, 0.05388913],
       [0.06482058, 0.05986862, 0.05977939, 0.05928425, 0.05624716],
       [0.07791546, 0.07015502, 0.05081303, 0.05076962, 0.05034688]],
      dtype=float32)

In [27]:
np.save("token_similarity_indices.npy", T)
np.save("token_similarity_values.npy", V)

In [6]:
T = np.load("token_similarity_indices.npy")
V = np.load("token_similarity_values.npy")

In [8]:
import numpy as np

def create_smooth_label_matrices(text, T, V, tokenizer, k=5, smooth_alpha=0.1, special_tokens=None):
    """
    Creates smoothed label matrices for a given text input based on token similarities.
    
    Args:
        text: Input text to tokenize
        T: Token similarity index matrix from find_similar_tokens (shape: [vocab_size, k])
        V: Token similarity value matrix from find_similar_tokens (shape: [vocab_size, k])
        tokenizer: The tokenizer used to convert text to token IDs
        k: Number of similar tokens used (must match the second dimension of T and V)
        smooth_alpha: Label smoothing hyperparameter (must match the one used to create V)
        special_tokens: List of token IDs for which to disable smoothing (default: None)
    
    Returns:
        L_index: Matrix with shape [n_tokens, k+1] where:
                 - L_index[i, 0] = original token ID
                 - L_index[i, 1:] = indices of the k most similar tokens
        L_values: Matrix with shape [n_tokens, k+1] where:
                 - L_values[i, 0] = 1-alpha (weight for the original token)
                 - L_values[i, 1:] = similarity values for the k similar tokens
    """
    # Ensure we have a list of special tokens
    if special_tokens is None:
        special_tokens = list(tokenizer.all_special_ids)
    
    # Get the end token ID (for GPT models it's typically eos_token_id)
    end_token_id = tokenizer.eos_token_id
    
    # Tokenize the input text
    tokens = tokenizer.encode(text, add_special_tokens=True)
    
    # Check if end token is present, add it if not
    if tokens[-1] != end_token_id:
        tokens.append(end_token_id)
    
    n_tokens = len(tokens)
    
    # Create matrices to store indices and values
    L_index = np.zeros((n_tokens, k+1), dtype=np.int32)
    L_values = np.zeros((n_tokens, k+1), dtype=np.float32)
    
    # Process each token in the input
    for i, token_id in enumerate(tokens):
        # Store the original token ID
        L_index[i, 0] = token_id
        
        if token_id in special_tokens:
            # For special tokens (like <END_TOKEN>), disable smoothing
            L_values[i, 0] = 1.0
            # Leave the rest as zeros (already initialized)
        else:
            # For regular tokens, add similar tokens
            L_index[i, 1:] = T[token_id]
            L_values[i, 0] = 1.0 - smooth_alpha  # Weight for the original token
            L_values[i, 1:] = V[token_id]        # Weights for similar tokens (should sum to alpha)
    
    return L_index, L_values

In [67]:
tokenizer.all_special_ids

[50256]

In [77]:
L_index, L_values=create_smooth_label_matrices(text="Shut the fuck upYoure a fucking cuntShut the fuck upYoure a stupid cunt, suck my dick", T=T, V=V, tokenizer=tokenizer, k=5, smooth_alpha=0.3, special_tokens=None)

In [78]:
L_values

array([[0.7       , 0.07853249, 0.05693075, 0.05588829, 0.05442713,
        0.05422135],
       [0.7       , 0.06838901, 0.06221119, 0.05993874, 0.05478674,
        0.05467436],
       [0.7       , 0.06737781, 0.06424569, 0.05780746, 0.05570666,
        0.0548624 ],
       [0.7       , 0.07983657, 0.05885753, 0.05497093, 0.05426022,
        0.05207478],
       [0.7       , 0.06566095, 0.06181142, 0.05870246, 0.05694684,
        0.05687837],
       [0.7       , 0.06923497, 0.06521451, 0.06437455, 0.05149898,
        0.049677  ],
       [0.7       , 0.079792  , 0.06813669, 0.05316404, 0.04954334,
        0.04936393],
       [0.7       , 0.06537926, 0.06230072, 0.05865724, 0.05837088,
        0.05529191],
       [0.7       , 0.06787285, 0.0588816 , 0.05790512, 0.05773303,
        0.05760743],
       [0.7       , 0.07853249, 0.05693075, 0.05588829, 0.05442713,
        0.05422135],
       [0.7       , 0.06838901, 0.06221119, 0.05993874, 0.05478674,
        0.05467436],
       [0.7       , 0

In [79]:
L_index

array([[39079, 49625, 34094, 19485, 21321, 46399],
       [  262,   257,   281,   383,   340,   428],
       [ 5089, 25617, 30998,  9372,  7510, 20654],
       [  510,  3205, 15958,   866, 19649,   503],
       [ 1639,  5832,  7120, 36981,  1532,  1858],
       [  260,  2200,   411,  3041, 21468, 34806],
       [  257,   281,   262,   617,   530,  1194],
       [ 9372, 43546, 46733, 30998, 35607,  5089],
       [40267, 18185, 29836, 21551, 33526, 40107],
       [39079, 49625, 34094, 19485, 21321, 46399],
       [  262,   257,   281,   383,   340,   428],
       [ 5089, 25617, 30998,  9372,  7510, 20654],
       [  510,  3205, 15958,   866, 19649,   503],
       [ 1639,  5832,  7120, 36981,  1532,  1858],
       [  260,  2200,   411,  3041, 21468, 34806],
       [  257,   281,   262,   617,   530,  1194],
       [ 8531, 14397, 19538, 13526, 42930, 37775],
       [40267, 18185, 29836, 21551, 33526, 40107],
       [   11,    13,    12,    26,    25,   338],
       [10110, 22523, 27904, 25

In [9]:
import numpy as np

def convert_ids_to_tokens(L_index, tokenizer):
    """
    Converts a matrix of token IDs to a matrix of token strings.
    
    Args:
        L_index: Matrix of token IDs (e.g., from create_smooth_label_matrices)
        tokenizer: The tokenizer used to convert IDs to tokens
    
    Returns:
        Array of the same shape as L_index containing token strings instead of IDs
    """
    # Get the shape of the input matrix
    shape = L_index.shape
    
    # Create an empty object array to store strings (using object dtype for variable-length strings)
    token_matrix = np.empty(shape, dtype=object)
    
    # Iterate through the matrix and convert each ID to its token
    for i in range(shape[0]):
        for j in range(shape[1]):
            token_id = L_index[i, j]
            # Only convert valid token IDs (skip zeros or padding)
            if token_id > 0:
                # Convert ID to token string
                token_matrix[i, j] = tokenizer.decode([token_id])
            else:
                token_matrix[i, j] = ""
    
    return token_matrix

In [80]:
convert_ids_to_tokens(L_index, tokenizer)

array([['Shut', 'shut', 'Fuck', 'Stop', 'Wait', 'Putting'],
       [' the', ' a', ' an', ' The', ' it', ' this'],
       [' fuck', ' Fuck', ' FUCK', ' fucking', ' shit', ' fucked'],
       [' up', ' Up', ' UP', ' down', ' ups', ' out'],
       ['You', 'you', 'Your', 'YOU', 'If', 'There'],
       ['re', 'RE', 'res', 'Re', 'rea', 'reb'],
       [' a', ' an', ' the', ' some', ' one', ' another'],
       [' fucking', ' goddamn', ' fuckin', ' FUCK', ' freaking', ' fuck'],
       [' cunt', ' pussy', ' asshole', ' bitch', ' slut', ' whore'],
       ['Shut', 'shut', 'Fuck', 'Stop', 'Wait', 'Putting'],
       [' the', ' a', ' an', ' The', ' it', ' this'],
       [' fuck', ' Fuck', ' FUCK', ' fucking', ' shit', ' fucked'],
       [' up', ' Up', ' UP', ' down', ' ups', ' out'],
       ['You', 'you', 'Your', 'YOU', 'If', 'There'],
       ['re', 'RE', 'res', 'Re', 'rea', 'reb'],
       [' a', ' an', ' the', ' some', ' one', ' another'],
       [' stupid', ' silly', ' foolish', ' dumb', ' Stupid',
 

In [63]:
import pandas as pd
data = pd.read_csv("./ASL_Citizen/resampled_renamed_df (Copy).csv")
data

,Participant ID,Video file,Gloss,Video Name,Frame Count,FPS,dom_hand,file_path,time_s
0,P1,15890366051589533-APPLE.mp4,APPLE,15890366051589533-APPLE,35,15.0,Right,./videos/15890366051589533-APPLE_fps15_R.mp4,2.373
1,P1,35618482303951104-IMPOSSIBLE.mp4,IMPOSSIBLE,35618482303951104-IMPOSSIBLE,38,15.0,Right,./videos/35618482303951104-IMPOSSIBLE_fps15_R.mp4,2.536
2,P1,6958143575951994-PARK.mp4,PARK,6958143575951994-PARK,43,15.0,Right,./videos/6958143575951994-PARK_fps15_R.mp4,2.900
3,P1,8006032738002744-SOCCER 2.mp4,SOCCER,8006032738002744-SOCCER 2,55,15.0,Right,./videos/8006032738002744-SOCCER 2_fps15_R.mp4,3.733
4,P1,37542279833186454-STINK.mp4,STINK,37542279833186454-STINK,43,15.0,Right,./videos/37542279833186454-STINK_fps15_R.mp4,2.900
...,...,...,...,...,...,...,...,...,...
81534,P22,782807099025346-WINDOW OPEN.mp4,WINDOWOPEN,782807099025346-WINDOW OPEN,17,15.0,Right,./videos/782807099025346-WINDOW OPEN_fps15_R.mp4,1.168
81535,P22,847315262724416-SPECIFIC.mp4,SPECIFIC,847315262724416-SPECIFIC,16,15.0,Right,./videos/847315262724416-SPECIFIC_fps15_R.mp4,1.067
81536,P22,7532911171254717-TAKE PILL.mp4,TAKEPILL,7532911171254717-TAKE PILL,7,15.0,Right,./videos/7532911171254717-TAKE PILL_fps15_R.mp4,0.533
81537,P22,4734112648368176-MEDITATE 2.mp4,MEDITATE,4734112648368176-MEDITATE 2,21,15.0,Right,./videos/4734112648368176-MEDITATE 2_fps15_R.mp4,1.401


In [61]:
import os

In [62]:
def find_landmark_directories(root_path):
    """
    Finds all directories with names ending in '_landmarks' under the given root path.
    
    Parameters:
    root_path (str): The path to the root directory to search in
    
    Returns:
    pandas.DataFrame: A DataFrame with one column 'landmarks_file_path' containing paths to landmark directories
    """
    landmark_paths = []
    
    # Walk through the directory structure
    for dirpath, dirnames, filenames in os.walk(root_path):
        # Check each directory name
        for dirname in dirnames:
            if dirname.endswith("_landmarks"):
                # Add the full path to our list
                dirname = dirname + f"/"
                full_path = os.path.join(dirpath, dirname)
                landmark_paths.append(full_path)
    
    # Create a DataFrame from the list of paths
    df = pd.DataFrame({'landmarks_file_path': landmark_paths})
    
    return df



In [87]:
data_1 = find_landmark_directories(root_path="./ASL_Citizen")

In [89]:

data_1['path'] = data_1['landmarks_file_path'].str.replace('ASL_Citizen', '.', regex=False)

In [92]:
data_1['path'].iloc[0]

'./videos/5864467858333573-STEAK_fps15_R_landmarks/'

In [91]:
data_1['path'] = data_1['path'].str.replace('././', './', regex=False)

In [69]:
data['path'] = data['file_path'].str.replace('.mp4', '_landmarks/', regex=False)

In [93]:
data['path'].iloc[0]

'./videos/15890366051589533-APPLE_fps15_R_landmarks/'

In [94]:
# With custom suffixes for duplicate columns
merged_data = pd.merge(data_1, data, on='path', how='inner', suffixes=('_df1', '_df2'))

In [95]:
merged_data_1 = merged_data.drop(['path'], axis=1)

In [96]:
merged_data_1

,landmarks_file_path,Participant ID,Video file,Gloss,Video Name,Frame Count,FPS,dom_hand,file_path,time_s
0,./ASL_Citizen/videos/5864467858333573-STEAK_fp...,P37,5864467858333573-STEAK.mp4,STEAK,5864467858333573-STEAK,29,15.0,Right,./videos/5864467858333573-STEAK_fps15_R.mp4,1.967000
1,./ASL_Citizen/videos/7610485274159868-BACKPACK...,P42,7610485274159868-BACKPACK.mp4,BACKPACK,7610485274159868-BACKPACK,21,15.0,Right,./videos/7610485274159868-BACKPACK_fps15_R.mp4,1.459002
2,./ASL_Citizen/videos/7144829404822042-SPILL 2_...,P39,7144829404822042-SPILL 2.mp4,SPILL,7144829404822042-SPILL 2,69,15.0,Right,./videos/7144829404822042-SPILL 2_fps15_R.mp4,4.633000
3,./ASL_Citizen/videos/4917408497989899-POPULAR_...,P21,4917408497989899-POPULAR.mp4,POPULAR,4917408497989899-POPULAR,24,15.0,Right,./videos/4917408497989899-POPULAR_fps15_R.mp4,1.633000
4,./ASL_Citizen/videos/8693330848301724-GROW_fps...,P15,8693330848301724-GROW.mp4,GROW,8693330848301724-GROW,46,15.0,Right,./videos/8693330848301724-GROW_fps15_R.mp4,3.067000
...,...,...,...,...,...,...,...,...,...,...
24885,./ASL_Citizen/videos/7474978008576432-TAKE PIL...,P42,7474978008576432-TAKE PILL.mp4,TAKEPILL,7474978008576432-TAKE PILL,29,15.0,Right,./videos/7474978008576432-TAKE PILL_fps15_R.mp4,1.959056
24886,./ASL_Citizen/videos/20766051469884816-STREET_...,P35,20766051469884816-STREET.mp4,STREET,20766051469884816-STREET,31,15.0,Right,./videos/20766051469884816-STREET_fps15_R.mp4,2.078000
24887,./ASL_Citizen/videos/09486030334348072-BUS_fps...,P18,09486030334348072-BUS.mp4,BUS,09486030334348072-BUS,36,15.0,Right,./videos/09486030334348072-BUS_fps15_R.mp4,2.400000
24888,./ASL_Citizen/videos/5324725711211808-MISUNDER...,P14,5324725711211808-MISUNDERSTAND.mp4,MISUNDERSTAND,5324725711211808-MISUNDERSTAND,24,15.0,Right,./videos/5324725711211808-MISUNDERSTAND_fps15_...,1.625012


In [97]:
# Convert all values in the "Gloss" column to lowercase
merged_data_1['Gloss'] = merged_data_1['Gloss'].str.lower()

In [98]:
merged_data_1

,landmarks_file_path,Participant ID,Video file,Gloss,Video Name,Frame Count,FPS,dom_hand,file_path,time_s
0,./ASL_Citizen/videos/5864467858333573-STEAK_fp...,P37,5864467858333573-STEAK.mp4,steak,5864467858333573-STEAK,29,15.0,Right,./videos/5864467858333573-STEAK_fps15_R.mp4,1.967000
1,./ASL_Citizen/videos/7610485274159868-BACKPACK...,P42,7610485274159868-BACKPACK.mp4,backpack,7610485274159868-BACKPACK,21,15.0,Right,./videos/7610485274159868-BACKPACK_fps15_R.mp4,1.459002
2,./ASL_Citizen/videos/7144829404822042-SPILL 2_...,P39,7144829404822042-SPILL 2.mp4,spill,7144829404822042-SPILL 2,69,15.0,Right,./videos/7144829404822042-SPILL 2_fps15_R.mp4,4.633000
3,./ASL_Citizen/videos/4917408497989899-POPULAR_...,P21,4917408497989899-POPULAR.mp4,popular,4917408497989899-POPULAR,24,15.0,Right,./videos/4917408497989899-POPULAR_fps15_R.mp4,1.633000
4,./ASL_Citizen/videos/8693330848301724-GROW_fps...,P15,8693330848301724-GROW.mp4,grow,8693330848301724-GROW,46,15.0,Right,./videos/8693330848301724-GROW_fps15_R.mp4,3.067000
...,...,...,...,...,...,...,...,...,...,...
24885,./ASL_Citizen/videos/7474978008576432-TAKE PIL...,P42,7474978008576432-TAKE PILL.mp4,takepill,7474978008576432-TAKE PILL,29,15.0,Right,./videos/7474978008576432-TAKE PILL_fps15_R.mp4,1.959056
24886,./ASL_Citizen/videos/20766051469884816-STREET_...,P35,20766051469884816-STREET.mp4,street,20766051469884816-STREET,31,15.0,Right,./videos/20766051469884816-STREET_fps15_R.mp4,2.078000
24887,./ASL_Citizen/videos/09486030334348072-BUS_fps...,P18,09486030334348072-BUS.mp4,bus,09486030334348072-BUS,36,15.0,Right,./videos/09486030334348072-BUS_fps15_R.mp4,2.400000
24888,./ASL_Citizen/videos/5324725711211808-MISUNDER...,P14,5324725711211808-MISUNDERSTAND.mp4,misunderstand,5324725711211808-MISUNDERSTAND,24,15.0,Right,./videos/5324725711211808-MISUNDERSTAND_fps15_...,1.625012


In [220]:
L_index, L_values=create_smooth_label_matrices(text=merged_data_1['Gloss'].iloc[0], T=T, V=V, tokenizer=tokenizer, k=5, smooth_alpha=0.3, special_tokens=None)

In [222]:
convert_ids_to_tokens(L_index, tokenizer)

array([['STE', 'Ste', 'ste', 'STEP', '\x18', '�'],
       ['AK', 'AKING', 'ak', 'aks', 'Ak', 'aky'],
       ['<|endoftext|>', '', '', '', '', '']], dtype=object)

In [226]:
L_values.sum(axis=1)

array([0.99999994, 1.        , 1.        ], dtype=float32)

In [99]:
import os
def save_smooth_labels_for_dataframe(dataframe, T, V, tokenizer, k=5, smooth_alpha=0.3, special_tokens=None, filename='smooth_labels.npz'):
    """
    For each row in the dataframe, generate and save L_index and L_values arrays to the specified directory.
    
    Args:
        dataframe: Pandas DataFrame containing 'landmarks_file_path' and 'Gloss' columns
        T: Token similarity index matrix
        V: Token similarity value matrix
        tokenizer: Tokenizer for converting text to token IDs
        k: Number of similar tokens to use
        smooth_alpha: Label smoothing hyperparameter
        special_tokens: List of token IDs for which to disable smoothing
        filename: Name of the npz file to save
    
    Returns:
        List of paths where the npz files were saved
    """
    # Check for required columns
    if 'landmarks_file_path' not in dataframe.columns or 'Gloss' not in dataframe.columns:
        raise ValueError("DataFrame must contain 'landmarks_file_path' and 'Gloss' columns")
    
    saved_files = []
    
    # Process each row in the dataframe
    for idx, row in tqdm(dataframe.iterrows(), total=len(dataframe), desc="Saving smooth labels"):
        directory_path = row['landmarks_file_path']
        text = row['Gloss']
        
        # Generate L_index and L_values matrices from the text
        L_index, L_values = create_smooth_label_matrices(
            text=text,
            T=T,
            V=V,
            tokenizer=tokenizer,
            k=k,
            smooth_alpha=smooth_alpha,
            special_tokens=special_tokens
        )
        
        # Create output path
        output_path = os.path.join(directory_path, filename)
        

        
        # Save the arrays and text label to an npz file
        np.savez(
            output_path,
            L_index=L_index,
            L_values=L_values,
            label=text
        )
        
        saved_files.append(output_path)
    
    print(f"Successfully saved {len(saved_files)} label files")
    return saved_files

In [100]:
saved_files = save_smooth_labels_for_dataframe(dataframe=merged_data_1, T=T, V=V, tokenizer=tokenizer, k=5, smooth_alpha=0.3, special_tokens=None, filename='smooth_labels.npz')

Saving smooth labels: 100%|██████████| 24890/24890 [00:08<00:00, 3009.85it/s]

Successfully saved 24890 label files


In [131]:
merged_data_1['landmarks_file_path'].iloc[4]

'./ASL_Citizen/videos/8693330848301724-GROW_fps15_R_landmarks/'

In [133]:
test_load = np.load("/home/thomasdev/Desktop/asl_project/ASL_Citizen/videos/8693330848301724-GROW_fps15_R_landmarks/smooth_labels.npz", allow_pickle=True)

# Print basic information

print(f"Arrays in file: {list(test_load.keys())}")

# Extract arrays
L_index = test_load['L_index']
L_values = test_load['L_values']
label = str(test_load['label']) 
convert_ids_to_tokens(L_index, tokenizer)

Arrays in file: ['L_index', 'L_values', 'label']


array([['grow', 'growing', 'grown', 'growth', 'Growing', '\x13'],
       ['<|endoftext|>', '', '', '', '', '']], dtype=object)

In [134]:
label

'grow'

In [17]:
import pandas as pd

In [18]:
data_open = find_landmark_directories(root_path="./OpenASL-main/Open_asl_all_data/clips")

In [29]:
data_open['path'] = data_open['landmarks_file_path']

In [21]:
data_open_full = pd.read_csv("./OpenASL-main/final_df.csv")
data_open_full

,word_count,tokenized-text,split,dom_hand,Video Name,Frame Count,file_path,clip_length_calculated,words_per_second
0,3,bush through relay,train,Right,RjJSxzMkC90-00:00:25.123-00:00:27.524_fps15,43,./Open_asl_all_data/clips/RjJSxzMkC90-00:00:25...,2.866667,1.046512
1,6,the secretary called me to come,train,Right,RjJSxzMkC90-00:00:14.083-00:00:17.094_fps15,55,./Open_asl_all_data/clips/RjJSxzMkC90-00:00:14...,3.666667,1.636364
2,5,i was in computer class,train,Right,RjJSxzMkC90-00:00:12.048-00:00:14.083_fps15,37,./Open_asl_all_data/clips/RjJSxzMkC90-00:00:12...,2.466667,2.027027
3,3,captioned by aslcaptions.com,train,Right,4SVP0ShrpN0-00:04:43.833-00:04:45.250_fps15,26,./Open_asl_all_data/clips/4SVP0ShrpN0-00:04:43...,1.733333,1.730769
4,8,see you tomorrow and stay with the light,train,Right,4SVP0ShrpN0-00:04:41.291-00:04:43.833_fps15,46,./Open_asl_all_data/clips/4SVP0ShrpN0-00:04:41...,3.066667,2.608696
...,...,...,...,...,...,...,...,...,...
29803,10,an armed security guard returned fire and kill...,test,Right,GhW-Vo0-w8k-00:03:55.266-00:04:01.433_fps15,88,./Open_asl_all_data/clips/GhW-Vo0-w8k-00:03:55...,5.866667,1.704545
29804,14,visitation was on december 3rd and 4th and the...,test,Right,GhW-Vo0-w8k-00:14:56.433-00:15:02.433_fps15,86,./Open_asl_all_data/clips/GhW-Vo0-w8k-00:14:56...,5.733333,2.441860
29805,7,roy passed away on friday from covid-19,test,Right,CkufhP_fqpQ-00:08:19.166-00:08:24.433_fps15,76,./Open_asl_all_data/clips/CkufhP_fqpQ-00:08:19...,5.066667,1.381579
29806,8,there are results pending for 20 other tests,test,Right,As6P9wKcyHE-00:03:30.600-00:03:36.566_fps15,87,./Open_asl_all_data/clips/As6P9wKcyHE-00:03:30...,5.800000,1.379310


In [23]:
data_open_full['file_path'].iloc[0]

'./Open_asl_all_data/clips/RjJSxzMkC90-00:00:25.123-00:00:27.524_fps15.mp4'

In [25]:
data_open_full['file_path'] = data_open_full['file_path'].str.replace('./', './OpenASL-main/', regex=False)
data_open_full['path'] = data_open_full['file_path'].str.replace('.mp4', '_landmarks/', regex=False)

In [30]:
merged_df = pd.merge(data_open_full, data_open, on='path', how='inner', suffixes=('_df1', '_df2'))

In [32]:
merged_df_1 = merged_df.drop(['path'], axis=1)

In [34]:
merged_df_1

,word_count,tokenized-text,split,dom_hand,Video Name,Frame Count,file_path,clip_length_calculated,words_per_second,landmarks_file_path
0,3,bush through relay,train,Right,RjJSxzMkC90-00:00:25.123-00:00:27.524_fps15,43,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...,2.866667,1.046512,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...
1,6,the secretary called me to come,train,Right,RjJSxzMkC90-00:00:14.083-00:00:17.094_fps15,55,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...,3.666667,1.636364,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...
2,5,i was in computer class,train,Right,RjJSxzMkC90-00:00:12.048-00:00:14.083_fps15,37,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...,2.466667,2.027027,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...
3,8,see you tomorrow and stay with the light,train,Right,4SVP0ShrpN0-00:04:41.291-00:04:43.833_fps15,46,./OpenASL-main/Open_asl_all_data/clips/4SVP0Sh...,3.066667,2.608696,./OpenASL-main/Open_asl_all_data/clips/4SVP0Sh...
4,7,check out our other videos on moth,train,Right,4SVP0ShrpN0-00:04:38.291-00:04:41.291_fps15,54,./OpenASL-main/Open_asl_all_data/clips/4SVP0Sh...,3.600000,1.944444,./OpenASL-main/Open_asl_all_data/clips/4SVP0Sh...
...,...,...,...,...,...,...,...,...,...,...
27447,10,an armed security guard returned fire and kill...,test,Right,GhW-Vo0-w8k-00:03:55.266-00:04:01.433_fps15,88,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...,5.866667,1.704545,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...
27448,14,visitation was on december 3rd and 4th and the...,test,Right,GhW-Vo0-w8k-00:14:56.433-00:15:02.433_fps15,86,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...,5.733333,2.441860,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...
27449,7,roy passed away on friday from covid-19,test,Right,CkufhP_fqpQ-00:08:19.166-00:08:24.433_fps15,76,./OpenASL-main/Open_asl_all_data/clips/CkufhP_...,5.066667,1.381579,./OpenASL-main/Open_asl_all_data/clips/CkufhP_...
27450,8,there are results pending for 20 other tests,test,Right,As6P9wKcyHE-00:03:30.600-00:03:36.566_fps15,87,./OpenASL-main/Open_asl_all_data/clips/As6P9wK...,5.800000,1.379310,./OpenASL-main/Open_asl_all_data/clips/As6P9wK...


In [55]:
L_index, L_values=create_smooth_label_matrices(text=merged_df_1['tokenized-text'].iloc[3], T=T, V=V, tokenizer=tokenizer, k=5, smooth_alpha=0.3, special_tokens=None)
convert_ids_to_tokens(L_index, tokenizer)

array([['see', 'See', 'SEE', 'remember', 'seen', 'although'],
       [' you', ' You', ' YOU', ' your', ' we', ' ya'],
       [' tomorrow', ' tonight', ' Tomorrow', ' yesterday', ' today',
        ' Yesterday'],
       [' and', ' or', ' &', ' in', ' but', ' with'],
       [' stay', ' stays', ' Stay', ' stayed', ' staying', ' remain'],
       [' with', ' With', ' in', ' on', ' for', ' WITH'],
       [' the', ' a', ' an', ' The', ' it', ' this'],
       [' light', ' Light', ' lights', ' lighting', ' lighter',
        ' sunlight'],
       ['<|endoftext|>', '', '', '', '', '']], dtype=object)

In [60]:
saved_files = save_smooth_labels_for_dataframe(dataframe=merged_df_1, T=T, V=V, tokenizer=tokenizer, k=5, smooth_alpha=0.3, special_tokens=None, filename='smooth_labels.npz')

Saving smooth labels: 100%|██████████| 27452/27452 [00:09<00:00, 2980.03it/s]

Successfully saved 27452 label files


In [147]:
test_load = np.load(f"{merged_df_1['landmarks_file_path'].iloc[123]}smooth_labels.npz", allow_pickle=True)



# Extract arrays
L_index = test_load['L_index']
L_values = test_load['L_values']
label = str(test_load['label']) 
print(convert_ids_to_tokens(L_index, tokenizer))
print(label)
print(L_values.sum(axis=1))


[['there' 'There' 'they' 'these' 'this' 'the']
 [' is' ' was' ' has' ' are' ' isn' ' does']
 [' one' ' One' ' two' ' ONE' ' a' ' ones']
 [' thing' ' things' ' Thing' ' stuff' ' Things' ' guy']
 [' that' ' That' ' the' ' it' ' THAT' ' this']
 [' i' ' ii' ' e' ' I' ' ie' ' iii']
 [' would' ' wouldn' ' could' ' Would' ' will' ' might']
 [' like' ' Like' ' LIKE' ' such' ' unlike' ' liked']
 [' to' ' for' ' To' ' in' ' on' ' with']
 [' share' ' shares' ' sharing' ' Share' ' shared' ' Sharing']
 ['<|endoftext|>' '' '' '' '' '']]
there is one thing that i would like to share
[1.         1.         0.99999994 0.99999994 0.99999994 1.
 1.0000001  1.         1.         1.         1.        ]


In [155]:
merged_df_1

,word_count,tokenized-text,split,dom_hand,Video Name,Frame Count,file_path,clip_length_calculated,words_per_second,landmarks_file_path
0,3,bush through relay,train,Right,RjJSxzMkC90-00:00:25.123-00:00:27.524_fps15,43,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...,2.866667,1.046512,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...
1,6,the secretary called me to come,train,Right,RjJSxzMkC90-00:00:14.083-00:00:17.094_fps15,55,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...,3.666667,1.636364,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...
2,5,i was in computer class,train,Right,RjJSxzMkC90-00:00:12.048-00:00:14.083_fps15,37,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...,2.466667,2.027027,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...
3,8,see you tomorrow and stay with the light,train,Right,4SVP0ShrpN0-00:04:41.291-00:04:43.833_fps15,46,./OpenASL-main/Open_asl_all_data/clips/4SVP0Sh...,3.066667,2.608696,./OpenASL-main/Open_asl_all_data/clips/4SVP0Sh...
4,7,check out our other videos on moth,train,Right,4SVP0ShrpN0-00:04:38.291-00:04:41.291_fps15,54,./OpenASL-main/Open_asl_all_data/clips/4SVP0Sh...,3.600000,1.944444,./OpenASL-main/Open_asl_all_data/clips/4SVP0Sh...
...,...,...,...,...,...,...,...,...,...,...
27447,10,an armed security guard returned fire and kill...,test,Right,GhW-Vo0-w8k-00:03:55.266-00:04:01.433_fps15,88,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...,5.866667,1.704545,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...
27448,14,visitation was on december 3rd and 4th and the...,test,Right,GhW-Vo0-w8k-00:14:56.433-00:15:02.433_fps15,86,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...,5.733333,2.441860,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...
27449,7,roy passed away on friday from covid-19,test,Right,CkufhP_fqpQ-00:08:19.166-00:08:24.433_fps15,76,./OpenASL-main/Open_asl_all_data/clips/CkufhP_...,5.066667,1.381579,./OpenASL-main/Open_asl_all_data/clips/CkufhP_...
27450,8,there are results pending for 20 other tests,test,Right,As6P9wKcyHE-00:03:30.600-00:03:36.566_fps15,87,./OpenASL-main/Open_asl_all_data/clips/As6P9wK...,5.800000,1.379310,./OpenASL-main/Open_asl_all_data/clips/As6P9wK...


In [160]:
merged_df_2 = merged_df_1.drop(['split', 'Video Name', 'file_path', 'clip_length_calculated'],axis=1)

In [161]:
merged_df_2

,word_count,tokenized-text,dom_hand,Frame Count,words_per_second,landmarks_file_path
0,3,bush through relay,Right,43,1.046512,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...
1,6,the secretary called me to come,Right,55,1.636364,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...
2,5,i was in computer class,Right,37,2.027027,./OpenASL-main/Open_asl_all_data/clips/RjJSxzM...
3,8,see you tomorrow and stay with the light,Right,46,2.608696,./OpenASL-main/Open_asl_all_data/clips/4SVP0Sh...
4,7,check out our other videos on moth,Right,54,1.944444,./OpenASL-main/Open_asl_all_data/clips/4SVP0Sh...
...,...,...,...,...,...,...
27447,10,an armed security guard returned fire and kill...,Right,88,1.704545,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...
27448,14,visitation was on december 3rd and 4th and the...,Right,86,2.441860,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...
27449,7,roy passed away on friday from covid-19,Right,76,1.381579,./OpenASL-main/Open_asl_all_data/clips/CkufhP_...
27450,8,there are results pending for 20 other tests,Right,87,1.379310,./OpenASL-main/Open_asl_all_data/clips/As6P9wK...


In [ ]:
merged_data_2 = merged_data_1.drop(['Participant ID', 'Video file', 'Video Name', 'FPS', 'file_path', 'time_s'], axis=1).rename(columns={'Gloss': 'tokenized-text'})
merged_data_2['word_count'] = 1

In [169]:
merged_data_2['words_per_second'] =15 / merged_data_2['Frame Count']

In [171]:
merged_data_2.drop(['words_per_second\t'], axis=1, inplace=True)

In [172]:
merged_data_2

,landmarks_file_path,tokenized-text,Frame Count,dom_hand,word_count,words_per_second
0,./ASL_Citizen/videos/5864467858333573-STEAK_fp...,steak,29,Right,1,0.517241
1,./ASL_Citizen/videos/7610485274159868-BACKPACK...,backpack,21,Right,1,0.714286
2,./ASL_Citizen/videos/7144829404822042-SPILL 2_...,spill,69,Right,1,0.217391
3,./ASL_Citizen/videos/4917408497989899-POPULAR_...,popular,24,Right,1,0.625000
4,./ASL_Citizen/videos/8693330848301724-GROW_fps...,grow,46,Right,1,0.326087
...,...,...,...,...,...,...
24885,./ASL_Citizen/videos/7474978008576432-TAKE PIL...,takepill,29,Right,1,0.517241
24886,./ASL_Citizen/videos/20766051469884816-STREET_...,street,31,Right,1,0.483871
24887,./ASL_Citizen/videos/09486030334348072-BUS_fps...,bus,36,Right,1,0.416667
24888,./ASL_Citizen/videos/5324725711211808-MISUNDER...,misunderstand,24,Right,1,0.625000


In [177]:
all_data_df = pd.concat([merged_data_2, merged_df_2], ignore_index=True)

In [182]:
all_data_df

,landmarks_file_path,tokenized-text,Frame Count,dom_hand,word_count,words_per_second
0,./ASL_Citizen/videos/5864467858333573-STEAK_fp...,steak,29,Right,1,0.517241
1,./ASL_Citizen/videos/7610485274159868-BACKPACK...,backpack,21,Right,1,0.714286
2,./ASL_Citizen/videos/7144829404822042-SPILL 2_...,spill,69,Right,1,0.217391
3,./ASL_Citizen/videos/4917408497989899-POPULAR_...,popular,24,Right,1,0.625000
4,./ASL_Citizen/videos/8693330848301724-GROW_fps...,grow,46,Right,1,0.326087
...,...,...,...,...,...,...
52337,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...,an armed security guard returned fire and kill...,88,Right,10,1.704545
52338,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...,visitation was on december 3rd and 4th and the...,86,Right,14,2.441860
52339,./OpenASL-main/Open_asl_all_data/clips/CkufhP_...,roy passed away on friday from covid-19,76,Right,7,1.381579
52340,./OpenASL-main/Open_asl_all_data/clips/As6P9wK...,there are results pending for 20 other tests,87,Right,8,1.379310


In [183]:
all_data_df.to_csv('all_data_df.csv', index=False)

In [184]:
def split_by_frame_count_tertiles(all_data_df):
    """
    Splits a DataFrame into three parts based on the 'Frame Count' percentiles.
    
    Parameters:
    all_data_df (pandas.DataFrame): DataFrame with a 'Frame Count' column
    
    Returns:
    tuple: Three DataFrames, each containing approximately 1/3 of rows based on 'Frame Count' values
    """
    # Sort the DataFrame by 'Frame Count'
    sorted_df = all_data_df.sort_values('Frame Count').reset_index(drop=True)
    
    # Calculate the size of each part
    total_rows = len(sorted_df)
    part_size = total_rows // 3
    
    # Split the DataFrame into three approximately equal parts
    low_tertile = sorted_df.iloc[:part_size].copy()
    mid_tertile = sorted_df.iloc[part_size:2*part_size].copy()
    high_tertile = sorted_df.iloc[2*part_size:].copy()
    
    print(f"Low tertile: {len(low_tertile)} rows, Frame Count range: {low_tertile['Frame Count'].min()} to {low_tertile['Frame Count'].max()}")
    print(f"Mid tertile: {len(mid_tertile)} rows, Frame Count range: {mid_tertile['Frame Count'].min()} to {mid_tertile['Frame Count'].max()}")
    print(f"High tertile: {len(high_tertile)} rows, Frame Count range: {high_tertile['Frame Count'].min()} to {high_tertile['Frame Count'].max()}")
    
    return low_tertile, mid_tertile, high_tertile


low_df, mid_df, high_df = split_by_frame_count_tertiles(all_data_df)

Low tertile: 17447 rows, Frame Count range: 13 to 38
Mid tertile: 17447 rows, Frame Count range: 38 to 60
High tertile: 17448 rows, Frame Count range: 60 to 119


In [202]:
high_df_only_path = high_df['landmarks_file_path']
mid_df_only_path = mid_df['landmarks_file_path']
low_df_only_path = low_df['landmarks_file_path']

In [203]:
low_df_only_path.to_csv('low_df_only_path.csv', index=False)
mid_df_only_path.to_csv('mid_df_only_path.csv', index=False)
high_df_only_path.to_csv('high_df_only_path.csv', index=False)

In [200]:
low_df.to_csv('low_df.csv', index=False)
mid_df.to_csv('mid_df.csv', index=False)
high_df.to_csv('high_df.csv', index=False)

In [207]:
high_df_only_path

34894    ./ASL_Citizen/videos/06298053274071624-RACECAR...
34895    ./OpenASL-main/Open_asl_all_data/clips/jfdwjt7...
34896    ./ASL_Citizen/videos/9724544998956868-NAVEL_fp...
34897    ./OpenASL-main/Open_asl_all_data/clips/MWmUdru...
34898    ./OpenASL-main/Open_asl_all_data/clips/4yQ5fvm...
                               ...                        
52337    ./OpenASL-main/Open_asl_all_data/clips/1k2s89d...
52338    ./OpenASL-main/Open_asl_all_data/clips/qmL-_ed...
52339    ./OpenASL-main/Open_asl_all_data/clips/HZ8H8pf...
52340    ./OpenASL-main/Open_asl_all_data/clips/6W1P4Y3...
52341    ./OpenASL-main/Open_asl_all_data/clips/Eu1VoQl...
Name: landmarks_file_path, Length: 17448, dtype: object

In [1]:
import numpy as np
import pandas as pd
import json
import sys 
import os
import glob
from tqdm import tqdm

In [4]:
data_all = pd.read_csv("./all_data_df.csv")

In [8]:
data_all.iloc[0]['landmarks_file_path']

'./ASL_Citizen/videos/5864467858333573-STEAK_fps15_R_landmarks/'

In [ ]:

def load_frame_data_with_velocities(npz_path):
    """
    Load saved frame data from an NPZ file.
    
    Args:
        npz_path (str): Path to the saved .npz file
        
    Returns:
        tuple: All the detection results for the frame
    """
    data = np.load(npz_path)
    
    # Extract all arrays from the npz file
    dom_landmarks = data['dom_landmarks']
    non_dom_landmarks = data['non_dom_landmarks']
    confidence_scores = data['confidence_scores']
    interpolation_scores = data['interpolation_scores']
    detection_status = data['detection_status']
    blendshape_scores = data['blendshape_scores']
    face_detected = data['face_detected'].item()  # Convert 0-d array to scalar
    nose_to_wrist_dist = data['nose_to_wrist_dist']
    frame_idx = data['frame_idx'].item()
    timestamp_ms = data['timestamp_ms'].item()
    dom_velocity_small = data['dom_velocity_small']
    dom_velocity_large = data['dom_velocity_large']
    non_dom_velocity_small = data['non_dom_velocity_small']
    non_dom_velocity_large = data['non_dom_velocity_large']
    velocity_confidence = data['velocity_confidence']
    velocity_calculation_confidence = data['velocity_calculation_confidence']
    nose_to_wrist_velocity_small = data['wrist_velocity_small']
    nose_to_wrist_velocity_large = data['wrist_velocity_large']
    
    return (dom_landmarks, non_dom_landmarks, confidence_scores, interpolation_scores,
            detection_status, blendshape_scores, face_detected, 
            nose_to_wrist_dist, frame_idx, timestamp_ms, dom_velocity_small, dom_velocity_large, non_dom_velocity_small, non_dom_velocity_large, velocity_confidence, velocity_calculation_confidence, nose_to_wrist_velocity_small, nose_to_wrist_velocity_large)


def load_frame_data_standardized(npz_path):
    """
    Load saved frame data from an NPZ file.
    
    Args:
        npz_path (str): Path to the saved .npz file
        
    Returns:
        tuple: All the detection results for the frame
    """
    data = np.load(npz_path)
    
    # Extract all arrays from the npz file
    dom_landmarks_standardized = data['dom_landmarks_standardized']
    non_dom_landmarks_standardized = data['non_dom_landmarks_standardized']
    confidence_scores = data['confidence_scores']
    interpolation_scores = data['interpolation_scores']
    detection_status = data['detection_status']
    blendshape_scores_standardized = data['blendshape_scores_standardized']
    face_detected = data['face_detected'].item()  # Convert 0-d array to scalar
    nose_to_wrist_dist_standardized = data['nose_to_wrist_dist_standardized']
    frame_idx = data['frame_idx'].item()
    timestamp_ms = data['timestamp_ms'].item()
    dom_velocity_small_standardized = data['dom_velocity_small_standardized']
    dom_velocity_large_standardized = data['dom_velocity_large_standardized']
    non_dom_velocity_small_standardized = data['non_dom_velocity_small_standardized']
    non_dom_velocity_large_standardized = data['non_dom_velocity_large_standardized']
    velocity_confidence = data['velocity_confidence']
    velocity_calculation_confidence = data['velocity_calculation_confidence']
    nose_to_wrist_velocity_small_standardized = data['wrist_velocity_small_standardized']
    nose_to_wrist_velocity_large_standardized = data['wrist_velocity_large_standardized']
    
    return (dom_landmarks_standardized, non_dom_landmarks_standardized, confidence_scores, interpolation_scores,
            detection_status, blendshape_scores_standardized, face_detected, 
            nose_to_wrist_dist_standardized, frame_idx, timestamp_ms, dom_velocity_small_standardized, dom_velocity_large_standardized, non_dom_velocity_small_standardized, non_dom_velocity_large_standardized, velocity_confidence, velocity_calculation_confidence, nose_to_wrist_velocity_small_standardized, nose_to_wrist_velocity_large_standardized)

def sorted_npz_files_checked(directory_path):
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        # List all NPZ files in the directory
        npz_files = sorted(glob.glob(os.path.join(directory_path, "*.npz")))
    else:
        print(f"Directory path {directory_path} doesn't exist or it isn't a directory")
        sys.exit(1)
        
    
    # Skip if no files found
    if not npz_files:
        print(f"No NPZ files found in {directory_path}")
        sys.exit(1)
    
    
    with open(os.path.join(directory_path, 'detection_statistics.json')) as f:
        statistics_file = json.load(f)
    
    if statistics_file['video_info']['total_frames'] != (len(npz_files)-1):
        print("npz filepath list contain different amount of items than total frames")
        sys.exit(1)


    frame_to_file = {}
    for file_path in npz_files:
        if os.path.basename(file_path) == 'smooth_labels.npz':
            continue
        try:
            frame_data = load_frame_data_with_velocities(file_path)
        except Exception as e:
            print(f"Error loading frame with path: {file_path}: {e}")
            sys.exit(1)
            
        frame_idx = frame_data[8]  # Index for frame_idx
        frame_to_file[frame_idx] = file_path

    
    frame_indices = sorted(frame_to_file.keys())
    if not all(frame_indices[i+1] - frame_indices[i] == 1 for i in range(len(frame_indices) - 1)):
        print("Consecutive frames are not different by one frame")
        sys.exit(1)

    

    return frame_to_file, frame_indices

In [36]:
frame_to_file, frame_indices = sorted_npz_files_checked(directory_path="./ASL_Citizen/videos/5864467858333573-STEAK_fps15_R_landmarks/")

In [37]:
frame_indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28]

In [38]:
for frame in frame_indices:
    dom_landmarks, non_dom_landmarks, confidence_scores, interpolation_scores,detection_status, blendshape_scores, face_detected, nose_to_wrist_dist, frame_idx, timestamp_ms, dom_velocity_small, dom_velocity_large, non_dom_velocity_small, non_dom_velocity_large, velocity_confidence, velocity_calculation_confidence, nose_to_wrist_velocity_small, nose_to_wrist_velocity_large = load_frame_data_with_velocities(frame_to_file[frame])

In [47]:
nose_to_wrist_velocity_small.shape

(2, 3)

In [41]:
np.zeros((20,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [53]:
dom_landmarks_mean=np.zeros((20,3))
non_dom_landmarks_mean=np.zeros((20,3))
blendshape_scores_mean = np.zeros((52,))
nose_to_wrist_dist_mean = np.zeros((2,2))
dom_velocity_small_mean = np.zeros((20, 5))
dom_velocity_large_mean = np.zeros((20, 5))
non_dom_velocity_small_mean = np.zeros((20, 5))
non_dom_velocity_large_mean=np.zeros((20, 5))
nose_to_wrist_velocity_small_mean=np.zeros((2, 3))
nose_to_wrist_velocity_large_mean=np.zeros((2, 3))
frames_loaded=0
total_videos=data_all.shape[0]
pbar = tqdm(total=total_videos, desc="Processing videos")
print(f"Processing {total_videos} videos for mean calculation")
for _, row in data_all.iterrows():
    dir_path = row['landmarks_file_path']
    frame_to_file, frame_indices = sorted_npz_files_checked(directory_path=dir_path)
    video_count += 1
    
    # Update the status line (stays on same line)
    video_name = os.path.basename(dir_path)
    pbar.set_description(f"Processing {video_name}")


    for frame in frame_indices:
        dom_landmarks, non_dom_landmarks, confidence_scores, interpolation_scores,detection_status, blendshape_scores, face_detected, nose_to_wrist_dist, frame_idx, timestamp_ms, dom_velocity_small, dom_velocity_large, non_dom_velocity_small, non_dom_velocity_large, velocity_confidence, velocity_calculation_confidence, nose_to_wrist_velocity_small, nose_to_wrist_velocity_large = load_frame_data_with_velocities(frame_to_file[frame])
        frames_loaded+=1
        dom_landmarks_mean+=dom_landmarks
        non_dom_landmarks_mean+=non_dom_landmarks
        blendshape_scores_mean+=blendshape_scores
        nose_to_wrist_dist_mean+=nose_to_wrist_dist
        dom_velocity_small_mean+=dom_velocity_small
        dom_velocity_large_mean+=dom_velocity_large
        non_dom_velocity_small_mean+=non_dom_velocity_small
        non_dom_velocity_large_mean+=non_dom_velocity_large
        nose_to_wrist_velocity_small_mean+=nose_to_wrist_velocity_small
        nose_to_wrist_velocity_large_mean+=nose_to_wrist_velocity_large
    pbar.update(1)

pbar.close()
dom_landmarks_mean = dom_landmarks_mean/frames_loaded
non_dom_landmarks_mean = non_dom_landmarks_mean/frames_loaded
blendshape_scores_mean = blendshape_scores_mean/frames_loaded
nose_to_wrist_dist_mean = nose_to_wrist_dist_mean/frames_loaded
dom_velocity_small_mean = dom_velocity_small_mean/frames_loaded
dom_velocity_large_mean = dom_velocity_large_mean/frames_loaded
non_dom_velocity_small_mean=non_dom_velocity_small_mean/frames_loaded
non_dom_velocity_large_mean=non_dom_velocity_large_mean/frames_loaded
nose_to_wrist_velocity_small_mean=nose_to_wrist_velocity_small_mean/frames_loaded
nose_to_wrist_velocity_large_mean=nose_to_wrist_velocity_large_mean/frames_loaded


Processing :   0%|          | 2/52342 [00:00<55:53, 15.61it/s]  

Processing 52342 videos for mean calculation


Processing : 100%|██████████| 52342/52342 [2:03:07<00:00,  7.09it/s]  


In [54]:

print(f"dom_landmarks_mean{dom_landmarks_mean}")

print(f"non_dom_landmarks_mean{non_dom_landmarks_mean}")

print(f"blendshape_scores_mean{blendshape_scores_mean}")

print(f"nose_to_wrist_dist_mean{nose_to_wrist_dist_mean}")

print(f"dom_velocity_small_mean{dom_velocity_small_mean}")

print(f"dom_velocity_large_mean{dom_velocity_large_mean}")

print(f"non_dom_velocity_small_mean{non_dom_velocity_small_mean}")

print(f"non_dom_velocity_large_mean{non_dom_velocity_large_mean}")

print(f"nose_to_wrist_velocity_small_mean{nose_to_wrist_velocity_small_mean}")

print(f"nose_to_wrist_velocity_large_mean{nose_to_wrist_velocity_large_mean}")


dom_landmarks_mean[[ 0.2423419  -0.33876453 -0.00475617]
 [ 0.507625   -0.67335777 -0.01031348]
 [ 0.70724671 -0.87932823 -0.0165875 ]
 [ 0.82745335 -1.00403451 -0.02251175]
 [ 0.49474303 -0.93272074 -0.0116978 ]
 [ 0.76552069 -1.14929651 -0.02341541]
 [ 0.90298584 -1.18298774 -0.03118425]
 [ 0.98744481 -1.19665466 -0.03570016]
 [ 0.44430206 -0.79815796 -0.01605627]
 [ 0.75220605 -0.94664942 -0.02668557]
 [ 0.86763943 -0.91550179 -0.03131958]
 [ 0.92049584 -0.88912868 -0.03395765]
 [ 0.38755015 -0.61043879 -0.0210505 ]
 [ 0.68366821 -0.71275999 -0.03101869]
 [ 0.77333979 -0.66630391 -0.03160314]
 [ 0.80398826 -0.6350235  -0.03086579]
 [ 0.33018091 -0.39607298 -0.0265289 ]
 [ 0.56506236 -0.47233681 -0.03304203]
 [ 0.6404405  -0.45782878 -0.0317866 ]
 [ 0.67052354 -0.44918624 -0.02999739]]
non_dom_landmarks_mean[[ 0.86413136  0.98254842 -0.00455193]
 [ 0.68450582  0.70518259 -0.00971607]
 [ 0.54364675  0.54303009 -0.0151607 ]
 [ 0.45677738  0.44842608 -0.02105114]
 [ 0.73002332  0.540642

In [55]:
dom_landmarks_std=np.zeros((20,3))
non_dom_landmarks_std=np.zeros((20,3))
blendshape_scores_std = np.zeros((52,))
nose_to_wrist_dist_std = np.zeros((2,2))
dom_velocity_small_std = np.zeros((20, 5))
dom_velocity_large_std = np.zeros((20, 5))
non_dom_velocity_small_std = np.zeros((20, 5))
non_dom_velocity_large_std=np.zeros((20, 5))
nose_to_wrist_velocity_small_std=np.zeros((2, 3))
nose_to_wrist_velocity_large_std=np.zeros((2, 3))
frames_loaded=0
total_videos=data_all.shape[0]
pbar = tqdm(total=total_videos, desc="Processing videos")
print(f"Processing {total_videos} videos for std calculation")
for _, row in data_all.iterrows():
    dir_path = row['landmarks_file_path']
    frame_to_file, frame_indices = sorted_npz_files_checked(directory_path=dir_path)

    
    # Update the status line (stays on same line)
    video_name = os.path.basename(dir_path)
    pbar.set_description(f"Processing {video_name}")


    for frame in frame_indices:
        dom_landmarks, non_dom_landmarks, confidence_scores, interpolation_scores,detection_status, blendshape_scores, face_detected, nose_to_wrist_dist, frame_idx, timestamp_ms, dom_velocity_small, dom_velocity_large, non_dom_velocity_small, non_dom_velocity_large, velocity_confidence, velocity_calculation_confidence, nose_to_wrist_velocity_small, nose_to_wrist_velocity_large = load_frame_data_with_velocities(frame_to_file[frame])
        frames_loaded+=1
        dom_landmarks_std+=np.square(dom_landmarks-dom_landmarks_mean)
        non_dom_landmarks_std+=np.square(non_dom_landmarks-non_dom_landmarks_mean)
        blendshape_scores_std+=np.square(blendshape_scores-blendshape_scores_mean)
        nose_to_wrist_dist_std+=np.square(nose_to_wrist_dist-nose_to_wrist_dist_mean)
        dom_velocity_small_std+=np.square(dom_velocity_small-dom_velocity_small_mean)
        dom_velocity_large_std+=np.square(dom_velocity_large-dom_velocity_large_mean)
        non_dom_velocity_small_std+=np.square(non_dom_velocity_small-non_dom_velocity_small_mean)
        non_dom_velocity_large_std+=np.square(non_dom_velocity_large-non_dom_velocity_large_mean)
        nose_to_wrist_velocity_small_std+=np.square(nose_to_wrist_velocity_small-nose_to_wrist_velocity_small_mean)
        nose_to_wrist_velocity_large_std+=np.square(nose_to_wrist_velocity_large-nose_to_wrist_velocity_large_mean)
    pbar.update(1)

pbar.close()
dom_landmarks_std = dom_landmarks_std/frames_loaded
non_dom_landmarks_std = non_dom_landmarks_std/frames_loaded
blendshape_scores_std = blendshape_scores_std/frames_loaded
nose_to_wrist_dist_std = nose_to_wrist_dist_std/frames_loaded
dom_velocity_small_std = dom_velocity_small_std/frames_loaded
dom_velocity_large_std = dom_velocity_large_std/frames_loaded
non_dom_velocity_small_std=non_dom_velocity_small_std/frames_loaded
non_dom_velocity_large_std=non_dom_velocity_large_std/frames_loaded
nose_to_wrist_velocity_small_std=nose_to_wrist_velocity_small_std/frames_loaded
nose_to_wrist_velocity_large_std=nose_to_wrist_velocity_large_std/frames_loaded

dom_landmarks_std = np.sqrt(dom_landmarks_std)
print(f"dom_landmarks_std{dom_landmarks_std}")
non_dom_landmarks_std = np.sqrt(non_dom_landmarks_std)
print(f"non_dom_landmarks_std{non_dom_landmarks_std}")
blendshape_scores_std = np.sqrt(blendshape_scores_std)
print(f"blendshape_scores_std{blendshape_scores_std}")
nose_to_wrist_dist_std = np.sqrt(nose_to_wrist_dist_std)
print(f"nose_to_wrist_dist_std{nose_to_wrist_dist_std}")
dom_velocity_small_std = np.sqrt(dom_velocity_small_std)
print(f"dom_velocity_small_std{dom_velocity_small_std}")
dom_velocity_large_std = np.sqrt(dom_velocity_large_std)
print(f"dom_velocity_large_std{dom_velocity_large_std}")
non_dom_velocity_small_std= np.sqrt(non_dom_velocity_small_std)
print(f"non_dom_velocity_small_std{non_dom_velocity_small_std}")
non_dom_velocity_large_std= np.sqrt(non_dom_velocity_large_std)
print(f"non_dom_velocity_large_std{non_dom_velocity_large_std}")
nose_to_wrist_velocity_small_std= np.sqrt(nose_to_wrist_velocity_small_std)
print(f"nose_to_wrist_velocity_small_std{nose_to_wrist_velocity_small_std}")
nose_to_wrist_velocity_large_std= np.sqrt(nose_to_wrist_velocity_large_std)
print(f"nose_to_wrist_velocity_large_std{nose_to_wrist_velocity_large_std}")


Processing :   0%|          | 1/52342 [00:00<2:01:58,  7.15it/s]

Processing 52342 videos for std calculation


Processing : 100%|██████████| 52342/52342 [2:03:29<00:00,  7.06it/s]  

dom_landmarks_std[[0.31659426 0.29394223 0.00801198]
 [0.55886241 0.56447972 0.01276103]
 [0.73860013 0.77933639 0.0169106 ]
 [0.88654163 0.94558944 0.02145128]
 [0.58920517 0.72755906 0.01690656]
 [0.81908635 1.01471785 0.0230818 ]
 [0.93978667 1.14924753 0.02684693]
 [1.03801658 1.26313896 0.02933976]
 [0.55991647 0.73213985 0.01695685]
 [0.81312179 1.01823253 0.02361478]
 [0.91116716 1.11929004 0.02569525]
 [0.99642683 1.20957993 0.0272212 ]
 [0.56774826 0.7222399  0.01784187]
 [0.8050297  0.97672196 0.02427248]
 [0.8734562  1.04187695 0.02480208]
 [0.93473114 1.10194077 0.02524577]
 [0.6158409  0.70586901 0.01991482]
 [0.80429932 0.90907861 0.02452906]
 [0.86099721 0.97439604 0.02476822]
 [0.91257278 1.03059846 0.02504025]]
non_dom_landmarks_std[[2.71923894 3.15639509 0.00819088]
 [2.76510666 3.15495194 0.01347469]
 [2.81774451 3.17335238 0.01816515]
 [2.86771513 3.19900812 0.02317179]
 [2.78296736 3.17392658 0.01618181]
 [2.84741523 3.22436202 0.02253845]
 [2.88304558 3.2621278  0

In [22]:
def modify_npz_file(file_path, modifications):
    """
    Load a .npz file, modify existing arrays and add new ones, then save it back.
    
    Args:
        file_path (str): Path to the .npz file
        modifications (dict): Dictionary with keys as array names and values as new arrays
                             or functions that take the original array and return a modified version
    """
    # Load the npz file
    with np.load(file_path) as data:
        # Create a copy of all arrays
        arrays = {name: data[name] for name in data.files}
    
    # Apply modifications and add new arrays
    for name, modification in modifications.items():
        if name in arrays:
            if callable(modification):
                # If the modification is a function, apply it to the original array
                arrays[name] = modification(arrays[name])
            else:
                # Otherwise, replace the array
                arrays[name] = modification
        else:
            # Add new array
            arrays[name] = modification

    
    # Save back to the file with same format
    np.savez(file_path, **arrays)

In [59]:

total_videos=data_all.shape[0]
pbar_mods = tqdm(total=total_videos, desc="Processing videos")
print(f"Processing {total_videos} videos for modifications")
for _, row in data_all.iterrows():
    dir_path = row['landmarks_file_path']
    frame_to_file, frame_indices = sorted_npz_files_checked(directory_path=dir_path)

    
    # Update the status line (stays on same line)
    video_name = os.path.basename(dir_path)
    pbar_mods.set_description(f"Processing {video_name}")
    for frame in frame_indices:
        dom_landmarks, non_dom_landmarks, confidence_scores, interpolation_scores,detection_status, blendshape_scores, face_detected, nose_to_wrist_dist, frame_idx, timestamp_ms, dom_velocity_small, dom_velocity_large, non_dom_velocity_small, non_dom_velocity_large, velocity_confidence, velocity_calculation_confidence, nose_to_wrist_velocity_small, nose_to_wrist_velocity_large = load_frame_data_with_velocities(frame_to_file[frame])
        new_dom_landmarks = (dom_landmarks-dom_landmarks_mean)/ dom_landmarks_std
        new_non_dom_landmarks=(non_dom_landmarks-non_dom_landmarks_mean)/non_dom_landmarks_std
        new_blendshape_scores=(blendshape_scores-blendshape_scores_mean)/blendshape_scores_std
        new_nose_to_wrist_dist=(nose_to_wrist_dist-nose_to_wrist_dist_mean)/nose_to_wrist_dist_std
        new_dom_velocity_small=(dom_velocity_small-dom_velocity_small_mean)/dom_velocity_small_std
        new_dom_velocity_large=(dom_velocity_large-dom_velocity_large_mean)/dom_velocity_large_std
        new_non_dom_velocity_small=(non_dom_velocity_small-non_dom_velocity_small_mean)/non_dom_velocity_small_std
        new_non_dom_velocity_large=(non_dom_velocity_large-non_dom_velocity_large_mean)/non_dom_velocity_large_std
        new_nose_to_wrist_velocity_small=(nose_to_wrist_velocity_small-nose_to_wrist_velocity_small_mean)/nose_to_wrist_velocity_small_std
        new_nose_to_wrist_velocity_large=(nose_to_wrist_velocity_large-nose_to_wrist_velocity_large_mean)/nose_to_wrist_velocity_large_std
        modifications = {
            'dom_landmarks_standardized': new_dom_landmarks,
            'non_dom_landmarks_standardized': new_non_dom_landmarks,
            'blendshape_scores_standardized': new_blendshape_scores,
            'nose_to_wrist_dist_standardized': new_nose_to_wrist_dist,
            'dom_velocity_small_standardized': new_dom_velocity_small,
            'dom_velocity_large_standardized': new_dom_velocity_large,
            'non_dom_velocity_small_standardized': new_non_dom_velocity_small,
            'non_dom_velocity_large_standardized': new_non_dom_velocity_large,
            'wrist_velocity_small_standardized': new_nose_to_wrist_velocity_small,
            'wrist_velocity_large_standardized': new_nose_to_wrist_velocity_large,
            }
        modify_npz_file(frame_to_file[frame], modifications)
    pbar_mods.update(1)

Processing 52342 videos for modifications


KeyboardInterrupt: 

In [16]:
for index, row in data_all.iloc[45000:].iterrows():
    print(f"row {index} is here")
    dir_path = row['landmarks_file_path']
    frame_to_file, frame_indices = sorted_npz_files_checked(directory_path=dir_path)
    try:
        lol=load_frame_data_standardized(frame_to_file[0])
    except Exception as e:
        print(f"Error reading {frame_to_file[0]}: {str(e)}")
        print(index)
        sys.exit(1)
        

row 45000 is here
row 45001 is here
row 45002 is here
row 45003 is here
row 45004 is here
row 45005 is here
row 45006 is here
row 45007 is here
row 45008 is here
row 45009 is here
row 45010 is here
row 45011 is here
row 45012 is here
row 45013 is here
row 45014 is here
row 45015 is here
row 45016 is here
row 45017 is here
row 45018 is here
row 45019 is here
row 45020 is here
row 45021 is here
row 45022 is here
row 45023 is here
row 45024 is here
row 45025 is here
row 45026 is here
row 45027 is here
row 45028 is here
row 45029 is here
row 45030 is here
row 45031 is here
row 45032 is here
row 45033 is here
row 45034 is here
row 45035 is here
row 45036 is here
row 45037 is here
row 45038 is here
row 45039 is here
row 45040 is here
row 45041 is here
row 45042 is here
row 45043 is here
row 45044 is here
row 45045 is here
row 45046 is here
row 45047 is here
row 45048 is here
row 45049 is here
row 45050 is here
row 45051 is here
row 45052 is here
row 45053 is here
row 45054 is here
row 45055 

SystemExit: 1

In [17]:
dom_landmarks_mean = np.array([
    [ 0.2423419,  -0.33876453, -0.00475617],
    [ 0.507625,   -0.67335777, -0.01031348],
    [ 0.70724671, -0.87932823, -0.0165875 ],
    [ 0.82745335, -1.00403451, -0.02251175],
    [ 0.49474303, -0.93272074, -0.0116978 ],
    [ 0.76552069, -1.14929651, -0.02341541],
    [ 0.90298584, -1.18298774, -0.03118425],
    [ 0.98744481, -1.19665466, -0.03570016],
    [ 0.44430206, -0.79815796, -0.01605627],
    [ 0.75220605, -0.94664942, -0.02668557],
    [ 0.86763943, -0.91550179, -0.03131958],
    [ 0.92049584, -0.88912868, -0.03395765],
    [ 0.38755015, -0.61043879, -0.0210505 ],
    [ 0.68366821, -0.71275999, -0.03101869],
    [ 0.77333979, -0.66630391, -0.03160314],
    [ 0.80398826, -0.6350235,  -0.03086579],
    [ 0.33018091, -0.39607298, -0.0265289 ],
    [ 0.56506236, -0.47233681, -0.03304203],
    [ 0.6404405,  -0.45782878, -0.0317866 ],
    [ 0.67052354, -0.44918624, -0.02999739]
])

non_dom_landmarks_mean = np.array([
    [ 0.86413136,  0.98254842, -0.00455193],
    [ 0.68450582,  0.70518259, -0.00971607],
    [ 0.54364675,  0.54303009, -0.0151607 ],
    [ 0.45677738,  0.44842608, -0.02105114],
    [ 0.73002332,  0.54064275, -0.01146802],
    [ 0.49183355,  0.39549388, -0.02127235],
    [ 0.36773076,  0.36955872, -0.02843994],
    [ 0.2898328,   0.35348602, -0.03297795],
    [ 0.76130194,  0.67389512, -0.01461407],
    [ 0.47548938,  0.58323956, -0.02287272],
    [ 0.3574962,   0.59954587, -0.02720896],
    [ 0.29700053,  0.60980237, -0.03045616],
    [ 0.78632099,  0.84380676, -0.01810058],
    [ 0.51335037,  0.77883619, -0.02574728],
    [ 0.41609126,  0.79776115, -0.0267518 ],
    [ 0.37227743,  0.80395597, -0.02722914],
    [ 0.80535341,  1.02743843, -0.02194167],
    [ 0.59041922,  0.98055168, -0.02768578],
    [ 0.50920375,  0.98243946, -0.02713891],
    [ 0.46951809,  0.97931604, -0.026194  ]
])

blendshape_scores_mean = np.array([
    2.07367799e-06, 1.12243794e-01, 1.30210855e-01, 1.40581997e-01,
    1.79121175e-01, 1.27687158e-01, 8.78810721e-05, 4.70528575e-07,
    4.94888999e-07, 2.89546712e-01, 2.53655051e-01, 4.31937720e-01,
    4.21590981e-01, 9.60258706e-02, 1.34417539e-01, 1.43088440e-01,
    1.02904453e-01, 4.03812221e-02, 3.79013323e-02, 3.41697851e-01,
    3.01469410e-01, 7.14214080e-03, 6.52515203e-03, 2.96341949e-04,
    7.16127947e-03, 4.90897988e-02, 8.56242324e-04, 4.30781488e-03,
    3.33408680e-02, 3.95012537e-02, 1.08384818e-02, 1.25596357e-02,
    7.82011760e-03, 1.43583489e-02, 2.60617592e-02, 3.06423700e-02,
    8.77857640e-02, 1.18501877e-01, 1.23479746e-01, 5.39464953e-03,
    9.61218425e-02, 8.40989499e-02, 1.56766318e-01, 5.27622142e-02,
    7.94297554e-02, 7.98463057e-02, 1.59781943e-02, 2.46436896e-02,
    2.84470992e-02, 3.67990232e-02, 1.90888781e-06, 1.24126707e-06
])

nose_to_wrist_dist_mean = np.array([
    [-1.45499613,  3.56198214],
    [ 1.05574818,  3.56466694]
])

dom_velocity_small_mean = np.array([
    [ 6.63706863e-02, -8.64027149e-04,  4.91705236e-03,  6.65556584e-01, 2.03861789e-05],
    [ 1.14049658e-01,  2.65335327e-03,  7.61210811e-03,  6.65901867e-01, -8.76681743e-05],
    [ 1.54478617e-01,  5.19758026e-03,  8.90128010e-03,  6.66030632e-01, -1.25373911e-04],
    [ 1.93029214e-01,  7.02277245e-03,  9.26182031e-03,  6.66005014e-01, -1.19373074e-04],
    [ 1.26442667e-01,  3.46382434e-03,  7.40864145e-03,  6.65459255e-01, -5.02119571e-04],
    [ 1.72929790e-01,  9.08913578e-03,  1.19949373e-02,  6.65444456e-01, -3.98243585e-04],
    [ 2.06398466e-01,  1.44529400e-02,  1.23423579e-02,  6.65601037e-01, -3.42448093e-04],
    [ 2.41116232e-01,  1.72336999e-02,  1.09914539e-02,  6.65779263e-01, -2.08363833e-04],
    [ 1.25521150e-01,  7.50826792e-03,  6.36430922e-03,  6.65599365e-01, -4.78748959e-04],
    [ 1.75603935e-01,  1.25519346e-02,  1.21903458e-02,  6.65470180e-01, -3.28210450e-04],
    [ 2.04632989e-01,  1.63630095e-02,  1.19889330e-02,  6.65731278e-01, -1.42016205e-04],
    [ 2.34104711e-01,  1.75157421e-02,  9.99214133e-03,  6.65920362e-01, -5.81938615e-05],
    [ 1.28516452e-01,  1.19213617e-02,  6.73975944e-03,  6.65693832e-01, -4.21968618e-04],
    [ 1.75971259e-01,  1.58661216e-02,  1.25249823e-02,  6.65678133e-01, -1.50421206e-04],
    [ 1.96083572e-01,  1.88808267e-02,  1.19369183e-02,  6.65896351e-01, 4.36452613e-05],
    [ 2.16258493e-01,  1.92396226e-02,  9.66445175e-03,  6.66045810e-01, 1.42029427e-04],
    [ 1.36118407e-01,  1.53079017e-02,  7.60457892e-03,  6.65655621e-01, -3.39297414e-04],
    [ 1.76287601e-01,  1.71744900e-02,  1.20819054e-02,  6.65740025e-01, -1.01866218e-04],
    [ 1.94025700e-01,  1.90868376e-02,  1.20271700e-02,  6.65841668e-01, 4.84755910e-05],
    [ 2.11932454e-01,  1.93487302e-02,  1.02378500e-02,  6.65923052e-01, 8.66735255e-05]
])

dom_velocity_large_mean = np.array([
    [ 2.43433283e-02, -5.42032176e-03,  3.90418740e-03,  6.23527343e-01, 2.89207586e-04],
    [ 4.20803675e-02,  1.45362055e-03,  5.97330740e-03,  6.23857031e-01, 3.92520370e-05],
    [ 5.67991829e-02,  6.11305862e-03,  6.88306804e-03,  6.24003513e-01, 3.90950424e-07],
    [ 7.07240175e-02,  9.89506059e-03,  7.36632332e-03,  6.24035114e-01, 3.84414778e-05],
    [ 4.60106982e-02,  5.83516490e-03,  3.49401071e-03,  6.23507014e-01, -6.96981592e-04],
    [ 6.41861150e-02,  1.31280230e-02,  8.37106277e-03,  6.23597248e-01, -5.66840336e-04],
    [ 7.65834775e-02,  1.89554649e-02,  9.34991941e-03,  6.23811459e-01, -3.15254194e-04],
    [ 8.86443894e-02,  2.14717184e-02,  9.11055845e-03,  6.23966205e-01, -1.34965580e-04],
    [ 4.64619846e-02,  1.14821636e-02,  1.67096592e-03,  6.23652278e-01, -7.92861273e-04],
    [ 6.66456765e-02,  1.71420533e-02,  9.63068413e-03,  6.23711142e-01, -5.65136833e-04],
    [ 7.73067706e-02,  2.05582131e-02,  1.01729836e-02,  6.23942214e-01, -2.58674269e-04],
    [ 8.75075488e-02,  2.13130281e-02,  8.59488141e-03,  6.24065347e-01, -4.87482868e-05],
    [ 4.92339850e-02,  1.67403304e-02,  2.97373427e-03,  6.23827748e-01, -7.36808080e-04],
    [ 6.82327297e-02,  2.10216109e-02,  1.05041050e-02,  6.23882146e-01, -4.24313270e-04],
    [ 7.54779010e-02,  2.31971842e-02,  1.02571866e-02,  6.24036947e-01, -1.06951485e-04],
    [ 8.23266883e-02,  2.28265668e-02,  8.36293897e-03,  6.24106104e-01, 4.43654935e-05],
    [ 5.37845766e-02,  1.93099174e-02,  4.89528479e-03,  6.23898846e-01, -6.13968103e-04],
    [ 6.94366742e-02,  2.21700043e-02,  1.03989093e-02,  6.23971354e-01, -2.55601023e-04],
    [ 7.55857173e-02,  2.39603834e-02,  1.04405909e-02,  6.24038125e-01, -1.09177351e-04],
    [ 8.14548656e-02,  2.36088452e-02,  9.18958086e-03,  6.24067272e-01, -4.07779853e-05]
])

non_dom_velocity_small_mean = np.array([
    [ 5.60008180e-01, -4.79768633e-04, -4.14456972e-03,  5.45467405e-01, 1.81741093e-05],
    [ 5.88497473e-01,  2.46264932e-03, -4.92494730e-03,  5.45638538e-01, 4.73865153e-05],
    [ 6.13699155e-01,  4.49145926e-03, -4.93990877e-03,  5.45686784e-01, 8.66302059e-05],
    [ 6.37464008e-01,  5.46397173e-03, -4.56188502e-03,  5.45682104e-01, 1.17404414e-04],
    [ 5.99374099e-01,  3.90704563e-03, -5.92142522e-03,  5.45463137e-01, -4.95225197e-05],
    [ 6.28871356e-01,  6.61490199e-03, -6.89849940e-03,  5.45426512e-01, 2.09388414e-06],
    [ 6.47353776e-01,  8.84086682e-03, -7.44100484e-03,  5.45464139e-01, 7.09044150e-05],
    [ 6.66946573e-01,  9.89732021e-03, -6.86484276e-03,  5.45536141e-01, 1.29634511e-04],
    [ 5.98796090e-01,  6.12983383e-03, -5.91739018e-03,  5.45546250e-01, -4.58434162e-05],
    [ 6.27859658e-01,  8.62583692e-03, -7.56283030e-03,  5.45451841e-01, 5.89697984e-05],
    [ 6.43200853e-01,  1.05133558e-02, -7.66761364e-03,  5.45538868e-01, 1.64625466e-04],
    [ 6.60360027e-01,  1.11201640e-02, -6.87010412e-03,  5.45646880e-01, 2.20012979e-04],
    [ 5.98658919e-01,  8.36861886e-03, -6.53698221e-03,  5.45584752e-01, 5.66261820e-05],
    [ 6.25248359e-01,  1.00312415e-02, -8.35647569e-03,  5.45509165e-01, 1.51135518e-04],
    [ 6.36875597e-01,  1.14568848e-02, -8.24403668e-03,  5.45612011e-01, 2.35861504e-04],
    [ 6.50236083e-01,  1.16520601e-02, -7.21999132e-03,  5.45728878e-01, 2.55700264e-04],
    [ 6.00060276e-01,  9.65750111e-03, -7.17543748e-03,  5.45491884e-01, 1.88292417e-04],
    [ 6.22278814e-01,  1.00036483e-02, -8.58998245e-03,  5.45491207e-01, 2.09443174e-04],
    [ 6.32477276e-01,  1.07243352e-02, -8.48247523e-03,  5.45514570e-01, 2.45915622e-04],
    [ 6.44004447e-01,  1.08201435e-02, -7.67620074e-03,  5.45597339e-01, 2.24344363e-04]
])

non_dom_velocity_large_mean = np.array([
    [ 1.75022277e-01, -1.00666712e-02, -7.50204833e-03,  5.15634614e-01, -5.14830233e-05],
    [ 1.86401452e-01, -5.78133705e-03, -7.70763762e-03,  5.15786322e-01, -5.24669396e-05],
    [ 1.96492842e-01, -3.04052413e-03, -7.70636126e-03,  5.15860101e-01, -5.73633984e-05],
    [ 2.05648391e-01, -1.08094351e-03, -7.57479989e-03,  5.15869035e-01, -9.57271395e-06],
    [ 1.91236656e-01, -2.36234407e-03, -7.77572567e-03,  5.15761277e-01, -1.24323217e-04],
    [ 2.03681592e-01,  5.60243976e-04, -8.79347072e-03,  5.15791573e-01, -3.30273531e-05],
    [ 2.10769187e-01,  2.83827547e-03, -9.02889624e-03,  5.15829461e-01, 7.83120422e-05],
    [ 2.17573312e-01,  3.53041714e-03, -8.83796017e-03,  5.15882681e-01, 1.37787026e-04],
    [ 1.91723824e-01,  3.36798506e-05, -7.45839369e-03,  5.15844958e-01, -7.08526906e-05],
    [ 2.04016102e-01,  2.78025645e-03, -9.23985049e-03,  5.15806944e-01, -8.25376282e-07],
    [ 2.09575462e-01,  4.44899223e-03, -9.09307723e-03,  5.15864244e-01, 1.20282041e-04],
    [ 2.15274224e-01,  4.79423840e-03, -8.41793076e-03,  5.15922236e-01, 1.65907626e-04],
    [ 1.92374515e-01,  1.92979641e-03, -8.24526896e-03,  5.15881267e-01, -1.31971655e-08],
    [ 2.03535825e-01,  4.06186676e-03, -1.01281346e-02,  5.15845646e-01, 6.13914079e-05],
    [ 2.07736597e-01,  5.31125529e-03, -9.79666089e-03,  5.15892569e-01, 1.71693241e-04],
    [ 2.12134046e-01,  5.35396023e-03, -8.81835493e-03,  5.15947463e-01, 1.77610100e-04],
    [ 1.93332261e-01,  2.62663888e-03, -8.97140058e-03,  5.15840433e-01, 1.33573473e-04],
    [ 2.02533776e-01,  3.75303788e-03, -1.04220871e-02,  5.15822682e-01, 1.70462952e-04],
    [ 2.06251244e-01,  4.67301195e-03, -1.01720668e-02,  5.15838861e-01, 2.31776537e-04],
    [ 2.10076687e-01,  4.62631820e-03, -9.38037108e-03,  5.15881627e-01, 2.12164170e-04]
])

nose_to_wrist_velocity_small_mean = np.array([
    [ 0.2962906,   0.02326938,  0.0011799 ],
    [ 0.25793079,  0.0169558,  -0.00530548]
])

nose_to_wrist_velocity_large_mean = np.array([
    [ 0.12304828,  0.01249294,  0.00173756],
    [ 0.10772689,  0.00666394, -0.00598551]
])

In [18]:

dom_landmarks_std = np.array([
    [0.31659426, 0.29394223, 0.00801198],
    [0.55886241, 0.56447972, 0.01276103],
    [0.73860013, 0.77933639, 0.0169106],
    [0.88654163, 0.94558944, 0.02145128],
    [0.58920517, 0.72755906, 0.01690656],
    [0.81908635, 1.01471785, 0.0230818],
    [0.93978667, 1.14924753, 0.02684693],
    [1.03801658, 1.26313896, 0.02933976],
    [0.55991647, 0.73213985, 0.01695685],
    [0.81312179, 1.01823253, 0.02361478],
    [0.91116716, 1.11929004, 0.02569525],
    [0.99642683, 1.20957993, 0.0272212],
    [0.56774826, 0.7222399, 0.01784187],
    [0.8050297, 0.97672196, 0.02427248],
    [0.8734562, 1.04187695, 0.02480208],
    [0.93473114, 1.10194077, 0.02524577],
    [0.6158409, 0.70586901, 0.01991482],
    [0.80429932, 0.90907861, 0.02452906],
    [0.86099721, 0.97439604, 0.02476822],
    [0.91257278, 1.03059846, 0.02504025]
])

non_dom_landmarks_std = np.array([
    [2.71923894, 3.15639509, 0.00819088],
    [2.76510666, 3.15495194, 0.01347469],
    [2.81774451, 3.17335238, 0.01816515],
    [2.86771513, 3.19900812, 0.02317179],
    [2.78296736, 3.17392658, 0.01618181],
    [2.84741523, 3.22436202, 0.02253845],
    [2.88304558, 3.2621278, 0.02702783],
    [2.91646864, 3.29984599, 0.03013556],
    [2.77812279, 3.18195064, 0.01627147],
    [2.84362182, 3.22684499, 0.02263396],
    [2.87079642, 3.25181444, 0.02553122],
    [2.89722214, 3.28020013, 0.02793461],
    [2.77684737, 3.18961276, 0.01721289],
    [2.83539535, 3.2262646, 0.0234064],
    [2.85427174, 3.24426206, 0.02462652],
    [2.87350044, 3.26519044, 0.02576783],
    [2.78051216, 3.19932568, 0.0192651],
    [2.82658657, 3.22569589, 0.02430904],
    [2.84277219, 3.23964186, 0.02483878],
    [2.85908182, 3.25508399, 0.02529234]
])

blendshape_scores_std = np.array([
    7.98018785e-05, 1.76607131e-01, 1.90191455e-01, 2.13094818e-01,
    2.18896492e-01, 1.82931443e-01, 1.98743232e-04, 7.92383467e-06,
    4.75419256e-06, 2.06869604e-01, 1.99485315e-01, 2.40078327e-01,
    2.34342282e-01, 1.27230898e-01, 1.39300709e-01, 1.40121028e-01,
    1.29151503e-01, 6.78961013e-02, 6.61083214e-02, 1.71472349e-01,
    1.61316617e-01, 1.35999699e-02, 1.49395266e-02, 7.60728688e-04,
    2.74996420e-02, 8.64166973e-02, 8.05163697e-03, 1.52468364e-02,
    8.35140719e-02, 9.92138501e-02, 3.71888926e-02, 4.21272633e-02,
    2.76062089e-02, 4.11323259e-02, 8.59522523e-02, 9.82525952e-02,
    1.19618786e-01, 1.45459750e-01, 2.51549966e-01, 2.51976657e-02,
    1.71056097e-01, 1.37772719e-01, 2.61652921e-01, 1.09795297e-01,
    1.85477210e-01, 1.85469815e-01, 5.06908860e-02, 6.61858183e-02,
    8.93761029e-02, 1.08801323e-01, 9.06385742e-06, 3.87205941e-06
])

nose_to_wrist_dist_std = np.array([
    [2.10186362, 2.5088656],
    [2.02219339, 2.84351392]
])

dom_velocity_small_std = np.array([
    [0.10484068, 0.5698322, 0.58538827, 0.47008614, 0.05887735],
    [0.18386131, 0.58017197, 0.57510746, 0.47027619, 0.05317981],
    [0.24799618, 0.58566444, 0.56947696, 0.47035148, 0.0508504],
    [0.30451061, 0.58519336, 0.56993571, 0.47033637, 0.05132346],
    [0.20656503, 0.57942021, 0.57586321, 0.47002845, 0.06041587],
    [0.29030206, 0.59241233, 0.56234707, 0.47002591, 0.06059909],
    [0.33804731, 0.60200688, 0.55194165, 0.47011367, 0.05814957],
    [0.38321893, 0.60267441, 0.55116135, 0.4702155, 0.05521236],
    [0.20534648, 0.58673429, 0.56838287, 0.47010421, 0.05824415],
    [0.29246716, 0.6018778, 0.55213228, 0.47004501, 0.06016734],
    [0.33482881, 0.60781544, 0.54549244, 0.47019007, 0.05600206],
    [0.37868843, 0.60260096, 0.55125178, 0.47029129, 0.05281343],
    [0.20894296, 0.58815377, 0.56683392, 0.47015879, 0.05670401],
    [0.28905911, 0.60143828, 0.5525183, 0.47015008, 0.05696132],
    [0.32041599, 0.60424197, 0.54936851, 0.47027202, 0.05328571],
    [0.35443273, 0.59810114, 0.55607963, 0.47036018, 0.05057028],
    [0.21897808, 0.5864819, 0.56847155, 0.47013658, 0.05733385],
    [0.28447269, 0.59612974, 0.55821276, 0.47018558, 0.05593583],
    [0.31197519, 0.59869815, 0.55539595, 0.4702422, 0.05422393],
    [0.3405355, 0.59401168, 0.56043201, 0.47028978, 0.05279298]
])

dom_velocity_large_std = np.array([
    [0.03220887, 0.5349144, 0.58196347, 0.48329709, 0.04997012],
    [0.05680787, 0.551824, 0.56596113, 0.48350333, 0.04343337],
    [0.0766087, 0.56193117, 0.55588501, 0.48359733, 0.04015916],
    [0.09378148, 0.56270089, 0.5550451, 0.48361301, 0.03947351],
    [0.06317162, 0.55297725, 0.56482689, 0.48328194, 0.05036472],
    [0.08986159, 0.57359236, 0.54369928, 0.48334233, 0.04864015],
    [0.10471874, 0.58695362, 0.5290541, 0.48347791, 0.0443599],
    [0.11784598, 0.59025389, 0.52527691, 0.48357556, 0.0409922],
    [0.06348365, 0.57035398, 0.54719396, 0.48337118, 0.04763432],
    [0.09152235, 0.5890558, 0.52676957, 0.48341536, 0.04640265],
    [0.10423163, 0.59823353, 0.51618798, 0.48356048, 0.04153109],
    [0.11658374, 0.59420995, 0.52081277, 0.48363709, 0.03869271],
    [0.0657179, 0.57310399, 0.54417117, 0.48348297, 0.04406973],
    [0.09107492, 0.58750823, 0.5283383, 0.48352299, 0.04284776],
    [0.10014131, 0.59242955, 0.52272692, 0.48361703, 0.03939503],
    [0.10959266, 0.58712544, 0.52872701, 0.4836622, 0.03770901],
    [0.06988247, 0.56639598, 0.55105198, 0.48352841, 0.04254],
    [0.09012869, 0.57765319, 0.53905164, 0.48357735, 0.04089194],
    [0.09781078, 0.58148338, 0.5348397, 0.48362225, 0.03931222],
    [0.10546851, 0.57787031, 0.53877973, 0.48364338, 0.03858294]
])

non_dom_velocity_small_std = np.array([
    [1.86339654, 0.52646172, 0.51907347, 0.49693261, 0.04625915],
    [1.86666701, 0.52827215, 0.51721836, 0.49705753, 0.0427669],
    [1.87172748, 0.52993326, 0.51550245, 0.49710274, 0.04161018],
    [1.87601915, 0.52958261, 0.51585678, 0.49709311, 0.04178616],
    [1.86493549, 0.52762024, 0.51786405, 0.49692306, 0.04641173],
    [1.87378369, 0.53224057, 0.5130743, 0.49690401, 0.04704189],
    [1.88020389, 0.53756344, 0.50745309, 0.49692788, 0.0463483],
    [1.88699862, 0.53812667, 0.50684438, 0.49699065, 0.04480191],
    [1.86341569, 0.53288206, 0.51242632, 0.49699651, 0.0446136],
    [1.87214782, 0.53873232, 0.50621389, 0.49692268, 0.04654842],
    [1.87744846, 0.54168741, 0.50301297, 0.49698742, 0.04480442],
    [1.88373931, 0.53779277, 0.50717324, 0.49707237, 0.04248655],
    [1.86228757, 0.53557215, 0.50957465, 0.49702046, 0.04386974],
    [1.87013408, 0.54083362, 0.50392971, 0.49697016, 0.04535418],
    [1.87437575, 0.54188001, 0.50277574, 0.49704538, 0.04324334],
    [1.87966116, 0.53677883, 0.50822948, 0.49713103, 0.04070968],
    [1.86201047, 0.53610733, 0.50898015, 0.49696058, 0.04566582],
    [1.86769579, 0.53966335, 0.5051794, 0.49694579, 0.04583439],
    [1.87035247, 0.54032572, 0.50445789, 0.49696783, 0.04531433],
    [1.87381683, 0.53584869, 0.50922176, 0.49703187, 0.04358247]
])

non_dom_velocity_large_std = np.array([
    [0.4453853, 0.50009284, 0.51588363, 0.49909301, 0.03759642],
    [0.44682544, 0.50725124, 0.50891034, 0.49921996, 0.03361972],
    [0.44878963, 0.51274811, 0.5033956, 0.49928209, 0.03149812],
    [0.45060372, 0.51402808, 0.50209857, 0.49929231, 0.03118839],
    [0.4481976, 0.51293722, 0.50320548, 0.49919769, 0.03432695],
    [0.45224197, 0.52261618, 0.49313394, 0.49922591, 0.03345043],
    [0.45466527, 0.52963266, 0.48557807, 0.49926219, 0.03230496],
    [0.45687708, 0.5310623, 0.48401307, 0.49930529, 0.03075142],
    [0.44779183, 0.52391449, 0.49177674, 0.49926995, 0.03193552],
    [0.4518592, 0.53311822, 0.481745, 0.49924426, 0.0329355],
    [0.45342322, 0.53625597, 0.47823994, 0.49929184, 0.03127478],
    [0.45502083, 0.53246528, 0.48246582, 0.49933593, 0.02956736],
    [0.44713614, 0.52747333, 0.48794115, 0.49929943, 0.03087047],
    [0.45078174, 0.53546197, 0.47911145, 0.49927599, 0.03182983],
    [0.45191156, 0.53540111, 0.47917414, 0.49931209, 0.03047371],
    [0.45312205, 0.52988203, 0.48528857, 0.49935544, 0.02878743],
    [0.44642484, 0.52570919, 0.48982532, 0.49926851, 0.0320308],
    [0.44929397, 0.53171785, 0.48325957, 0.49925637, 0.03250224],
    [0.45032419, 0.53121382, 0.48381087, 0.49926842, 0.0320569],
    [0.45131858, 0.52610233, 0.48938055, 0.49930276, 0.03080967]
])

nose_to_wrist_velocity_small_std = np.array([
    [0.81310188, 0.60711879, 0.54613179],
    [0.82848419, 0.5521718, 0.49124275]
])

nose_to_wrist_velocity_large_std = np.array([
    [0.23908125, 0.58171989, 0.53507233],
    [0.22367836, 0.52994637, 0.48524529]
])

In [23]:

total_videos=data_all.shape[0]
pbar_mods = tqdm(total=total_videos, desc="Processing videos")
print(f"Processing {total_videos} videos for modifications")
for _, row in data_all.iloc[45577:].iterrows():
    dir_path = row['landmarks_file_path']
    frame_to_file, frame_indices = sorted_npz_files_checked(directory_path=dir_path)

    
    # Update the status line (stays on same line)
    video_name = os.path.basename(dir_path)
    pbar_mods.set_description(f"Processing {video_name}")
    for frame in frame_indices:
        dom_landmarks, non_dom_landmarks, confidence_scores, interpolation_scores,detection_status, blendshape_scores, face_detected, nose_to_wrist_dist, frame_idx, timestamp_ms, dom_velocity_small, dom_velocity_large, non_dom_velocity_small, non_dom_velocity_large, velocity_confidence, velocity_calculation_confidence, nose_to_wrist_velocity_small, nose_to_wrist_velocity_large = load_frame_data_with_velocities(frame_to_file[frame])
        new_dom_landmarks = (dom_landmarks-dom_landmarks_mean)/ dom_landmarks_std
        new_non_dom_landmarks=(non_dom_landmarks-non_dom_landmarks_mean)/non_dom_landmarks_std
        new_blendshape_scores=(blendshape_scores-blendshape_scores_mean)/blendshape_scores_std
        new_nose_to_wrist_dist=(nose_to_wrist_dist-nose_to_wrist_dist_mean)/nose_to_wrist_dist_std
        new_dom_velocity_small=(dom_velocity_small-dom_velocity_small_mean)/dom_velocity_small_std
        new_dom_velocity_large=(dom_velocity_large-dom_velocity_large_mean)/dom_velocity_large_std
        new_non_dom_velocity_small=(non_dom_velocity_small-non_dom_velocity_small_mean)/non_dom_velocity_small_std
        new_non_dom_velocity_large=(non_dom_velocity_large-non_dom_velocity_large_mean)/non_dom_velocity_large_std
        new_nose_to_wrist_velocity_small=(nose_to_wrist_velocity_small-nose_to_wrist_velocity_small_mean)/nose_to_wrist_velocity_small_std
        new_nose_to_wrist_velocity_large=(nose_to_wrist_velocity_large-nose_to_wrist_velocity_large_mean)/nose_to_wrist_velocity_large_std
        modifications = {
            'dom_landmarks_standardized': new_dom_landmarks,
            'non_dom_landmarks_standardized': new_non_dom_landmarks,
            'blendshape_scores_standardized': new_blendshape_scores,
            'nose_to_wrist_dist_standardized': new_nose_to_wrist_dist,
            'dom_velocity_small_standardized': new_dom_velocity_small,
            'dom_velocity_large_standardized': new_dom_velocity_large,
            'non_dom_velocity_small_standardized': new_non_dom_velocity_small,
            'non_dom_velocity_large_standardized': new_non_dom_velocity_large,
            'wrist_velocity_small_standardized': new_nose_to_wrist_velocity_small,
            'wrist_velocity_large_standardized': new_nose_to_wrist_velocity_large,
            }
        modify_npz_file(frame_to_file[frame], modifications)
    pbar_mods.update(1)

Processing :   0%|          | 0/52342 [00:09<?, ?it/s]


Processing 52342 videos for modifications


In [ ]:
data_all.iloc

,landmarks_file_path,tokenized-text,Frame Count,dom_hand,word_count,words_per_second
0,./ASL_Citizen/videos/5864467858333573-STEAK_fp...,steak,29,Right,1,0.517241
1,./ASL_Citizen/videos/7610485274159868-BACKPACK...,backpack,21,Right,1,0.714286
2,./ASL_Citizen/videos/7144829404822042-SPILL 2_...,spill,69,Right,1,0.217391
3,./ASL_Citizen/videos/4917408497989899-POPULAR_...,popular,24,Right,1,0.625000
4,./ASL_Citizen/videos/8693330848301724-GROW_fps...,grow,46,Right,1,0.326087
...,...,...,...,...,...,...
52337,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...,an armed security guard returned fire and kill...,88,Right,10,1.704545
52338,./OpenASL-main/Open_asl_all_data/clips/GhW-Vo0...,visitation was on december 3rd and 4th and the...,86,Right,14,2.441860
52339,./OpenASL-main/Open_asl_all_data/clips/CkufhP_...,roy passed away on friday from covid-19,76,Right,7,1.381579
52340,./OpenASL-main/Open_asl_all_data/clips/As6P9wK...,there are results pending for 20 other tests,87,Right,8,1.379310


In [ ]:
def sorted_npz_files_checked_label(directory_path):
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        # List all NPZ files in the directory
        npz_files = sorted(glob.glob(os.path.join(directory_path, "*.npz")))
    else:
        print(f"Directory path {directory_path} doesn't exist or it isn't a directory")
        sys.exit(1)
        
    
    # Skip if no files found
    if not npz_files:
        print(f"No NPZ files found in {directory_path}")
        sys.exit(1)
    
    
    with open(os.path.join(directory_path, 'detection_statistics.json')) as f:
        statistics_file = json.load(f)
    
    if statistics_file['video_info']['total_frames'] != (len(npz_files)-1):
        print("npz filepath list contain different amount of items than total frames")
        sys.exit(1)


    frame_to_file = {}
    for file_path in npz_files:
        if os.path.basename(file_path) == 'smooth_labels.npz':
            label_path = file_path
            continue
        try:
            frame_data = load_frame_data_with_velocities(file_path)
        except Exception as e:
            print(f"Error loading frame with path: {file_path}: {e}")
            sys.exit(1)
            
        frame_idx = frame_data[8]  # Index for frame_idx
        frame_to_file[frame_idx] = file_path

    
    frame_indices = sorted(frame_to_file.keys())
    if not all(frame_indices[i+1] - frame_indices[i] == 1 for i in range(len(frame_indices) - 1)):
        print("Consecutive frames are not different by one frame")
        sys.exit(1)

    

    return frame_to_file, frame_indices, label_path

In [1]:
import pandas as pd

In [2]:
high_df_full = pd.read_csv("./high_df.csv")
mid_df_full = pd.read_csv("./mid_df.csv")
low_df_full = pd.read_csv("./low_df.csv")

In [3]:
high_df_full['word_count']

0         1
1         2
2         1
3         5
4        11
         ..
17443    11
17444    15
17445    15
17446    15
17447    13
Name: word_count, Length: 17448, dtype: int64

In [4]:
mid_df_full

,landmarks_file_path,tokenized-text,Frame Count,dom_hand,word_count,words_per_second
0,./ASL_Citizen/videos/6337882711799459-WHEEL_fp...,wheel,38,Right,1,0.394737
1,./ASL_Citizen/videos/8951093496358717-RECORDIN...,recording,38,Right,1,0.394737
2,./ASL_Citizen/videos/013132284306153874-TOTAL_...,total,38,Right,1,0.394737
3,./OpenASL-main/Open_asl_all_data/clips/j8N8RXx...,congratulations to her for making history,38,Right,6,2.368421
4,./ASL_Citizen/videos/395104563048577-CAROUSEL_...,carousel,38,Right,1,0.394737
...,...,...,...,...,...,...
17442,./ASL_Citizen/videos/752482831606313-CHOKE_fps...,choke,60,Right,1,0.250000
17443,./OpenASL-main/Open_asl_all_data/clips/RY00i2N...,hello i 'm melissa,60,Right,4,1.000000
17444,./OpenASL-main/Open_asl_all_data/clips/sgmdxup...,you are suffering from veisalgia,60,Right,5,1.250000
17445,./OpenASL-main/Open_asl_all_data/clips/gdVQKVc...,the detective said it was his first time with ...,60,Right,10,2.500000


In [5]:
low_df_full

,landmarks_file_path,tokenized-text,Frame Count,dom_hand,word_count,words_per_second
0,./ASL_Citizen/videos/5278240556504277-FOUR_fps...,four,13,Left,1,1.153846
1,./ASL_Citizen/videos/32791810480357064-THREE_f...,three,13,Left,1,1.153846
2,./OpenASL-main/Open_asl_all_data/clips/lYv-muF...,we can hope,14,Right,3,3.214286
3,./OpenASL-main/Open_asl_all_data/clips/t-krQnP...,oh yeah,14,Right,2,2.142857
4,./OpenASL-main/Open_asl_all_data/clips/HyGIBdT...,see this,14,Right,2,2.142857
...,...,...,...,...,...,...
17442,./ASL_Citizen/videos/09255709495802078-PLOW 2_...,plow,38,Right,1,0.394737
17443,./ASL_Citizen/videos/15471453286217618-ANGRY_f...,angry,38,Right,1,0.394737
17444,./OpenASL-main/Open_asl_all_data/clips/4i0Y_mF...,then suddenly your symptoms come back,38,Right,6,2.368421
17445,./ASL_Citizen/videos/6634664399874755-TOW 1_fp...,tow,38,Right,1,0.394737


In [6]:
from sklearn.model_selection import train_test_split

# For each dataframe
for name, df in {'high': high_df_full, 'mid': mid_df_full, 'low': low_df_full}.items():
    # First split: separate out the test set (10%)
    df_temp, df_test = train_test_split(
        df, 
        test_size=0.1,
        random_state=42,
        stratify=df['word_count']
    )
    
    # Second split: from the remaining 90%, get validation set (~11.11% of the remainder)
    df_train, df_val = train_test_split(
        df_temp,
        test_size=0.1/0.9,  # This ensures val is 10% of original dataset
        random_state=42,
        stratify=df_temp['word_count']
    )
    
    # Save the resulting dataframes
    df_train.to_csv(f'{name}_train.csv', index=False)
    df_val.to_csv(f'{name}_val.csv', index=False)
    df_test.to_csv(f'{name}_test.csv', index=False)
    
    # Print sizes to verify the split
    print(f"{name} splits - Original: {len(df)}, Train: {len(df_train)} ({len(df_train)/len(df):.1%}), "
          f"Val: {len(df_val)} ({len(df_val)/len(df):.1%}), Test: {len(df_test)} ({len(df_test)/len(df):.1%})")

high splits - Original: 17448, Train: 13958 (80.0%), Val: 1745 (10.0%), Test: 1745 (10.0%)
mid splits - Original: 17447, Train: 13957 (80.0%), Val: 1745 (10.0%), Test: 1745 (10.0%)
low splits - Original: 17447, Train: 13957 (80.0%), Val: 1745 (10.0%), Test: 1745 (10.0%)


In [7]:
high_train_df = pd.read_csv("./high_train.csv")
high_val_df = pd.read_csv("./high_val.csv")
high_test_df = pd.read_csv("./high_test.csv")

mid_train_df = pd.read_csv("./mid_train.csv")
mid_val_df = pd.read_csv("./mid_val.csv")
mid_test_df = pd.read_csv("./mid_test.csv")

low_train_df = pd.read_csv("./low_train.csv")
low_val_df = pd.read_csv("./low_val.csv")
low_test_df = pd.read_csv("./low_test.csv")

In [11]:
# For high proficiency dataframes
high_train_only_path = high_train_df[['landmarks_file_path']]
high_val_only_path = high_val_df[['landmarks_file_path']]
high_test_only_path = high_test_df[['landmarks_file_path']]

# Save high proficiency dataframes with new naming convention
high_train_only_path.to_csv("./high_train_only_path.csv", index=False)
high_val_only_path.to_csv("./high_val_only_path.csv", index=False)
high_test_only_path.to_csv("./high_test_only_path.csv", index=False)

# For mid proficiency dataframes
mid_train_only_path = mid_train_df[['landmarks_file_path']]
mid_val_only_path = mid_val_df[['landmarks_file_path']]
mid_test_only_path = mid_test_df[['landmarks_file_path']]

# Save mid proficiency dataframes with new naming convention
mid_train_only_path.to_csv("./mid_train_only_path.csv", index=False)
mid_val_only_path.to_csv("./mid_val_only_path.csv", index=False)
mid_test_only_path.to_csv("./mid_test_only_path.csv", index=False)

# For low proficiency dataframes
low_train_only_path = low_train_df[['landmarks_file_path']]
low_val_only_path = low_val_df[['landmarks_file_path']]
low_test_only_path = low_test_df[['landmarks_file_path']]

# Save low proficiency dataframes with new naming convention
low_train_only_path.to_csv("./low_train_only_path.csv", index=False)
low_val_only_path.to_csv("./low_val_only_path.csv", index=False)
low_test_only_path.to_csv("./low_test_only_path.csv", index=False)

# Print confirmation
print("All dataframes with only 'landmarks_file_path' column have been saved.")

All dataframes with only 'landmarks_file_path' column have been saved.
